In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std

from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import mode

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [ ]:
!pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 5.2 MB/s 


In [ ]:
#import pyeeg
from tensorflow import keras 

In [ ]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [ ]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings 
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [ ]:
Fs = 100
from keras.utils import np_utils

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way.csv',header=None)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_plus_sub_dt_with_tri_trained_labels_keras_way_sub_473_trial_2.csv',header=None) #the transfer set + subject specific train data , inc semi sup tri-trained labels obtained the keras way (train_on_batch)

In [ ]:
transfer_set_with_tri_trained_sub_labeled_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120701,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
120702,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
120703,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
120704,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,2


In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 5.1 MB/s 


In [ ]:
x_dt=transfer_set_with_tri_trained_sub_labeled_dt.drop([3000],axis=1)
y_dt=transfer_set_with_tri_trained_sub_labeled_dt[3000]

In [ ]:
y_dt_=np_utils.to_categorical(y_dt)

In [ ]:
y_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
x_dt.shape

(120706, 3000)

In [ ]:
x_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120701,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,0.004933,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379
120702,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.022106,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632
120703,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.000567,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397
120704,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.039062,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846


In [ ]:
x_dt_rshp=np.array(x_dt).reshape(120706,3000,1)

In [ ]:
x_dt_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.02463891],
        [ 0.03105487],
        [ 0.03884568],
        ...,
        [-0.0078992 ],
        [-0.01523173],
        [-0.02439739]],

       [[-0.01431516],
        [-0.05601893],
        [-0.06793429],
        ...,
        [-0.11238776],
        [-0.1059718 ],
        [-0.11284604]],

       [[-0.18449727],
        [-0.17117053],
        [-0.17164648],
        ...,
        [ 0.00921632],
        [ 0.01112014],
        [ 0.00112509]]])

In [ ]:
sub_test_data=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [ ]:
sub_test_data.shape

(997, 3001)

In [ ]:
x_test_dt=sub_test_data.drop([3000],axis=1)
y_test_dt=sub_test_data[3000]

In [ ]:
y_test_dt_=np_utils.to_categorical(y_test_dt)

In [ ]:
y_test_dt_

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.045043,-0.060749,-0.036475,-0.148801,-0.090258,-0.033144,-0.089782,-0.091686,-0.106441,-0.140709,...,0.025399,0.018735,-0.010774,-0.001731,0.049197,0.061095,0.037774,0.085845,0.038250,0.019211
1,0.047095,0.051678,0.048011,0.055344,0.047095,0.053969,0.051219,0.007224,0.011807,0.089257,...,-0.113763,0.085591,-0.123845,0.060843,-0.138510,-0.149050,0.010890,-0.095431,-0.128428,0.000808
2,0.039304,0.029680,0.024639,0.030597,0.038387,0.036554,0.050761,0.059468,0.050761,0.046178,...,-0.147676,-0.126136,-0.138968,-0.127053,-0.122928,-0.116512,-0.115137,-0.115596,-0.124761,-0.114679
3,-0.050278,-0.048850,-0.049326,-0.055038,-0.052658,-0.051230,-0.035999,-0.028384,-0.012678,-0.006966,...,-0.003159,-0.013154,-0.013630,-0.010298,-0.006966,-0.005062,-0.004110,-0.013154,-0.029812,-0.042663
4,-0.077558,-0.068851,-0.067934,-0.068851,-0.067934,-0.072059,-0.071142,-0.068851,-0.071142,-0.060602,...,-0.027605,-0.026231,-0.032188,-0.032188,-0.040437,-0.044562,-0.038604,-0.036313,-0.038146,-0.016607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,-0.021721,-0.017437,-0.023625,-0.030288,-0.035999,-0.023625,-0.003634,0.014452,0.014928,0.011120,...,0.004933,-0.001255,0.003505,-0.003634,-0.007918,-0.013154,-0.012678,-0.020769,-0.018389,-0.014105
993,0.015015,0.013640,0.007224,0.016848,0.022806,0.027847,0.031971,0.036554,0.024181,0.009516,...,0.017306,0.032430,0.026472,0.007682,0.004474,0.023722,0.042512,0.027847,0.027847,0.005391
994,-0.006983,-0.001483,-0.001942,-0.007899,0.000350,0.002183,0.001266,-0.006983,0.000350,0.004933,...,0.029222,0.007682,0.011349,0.020973,0.013640,0.007224,-0.001025,-0.006983,-0.011107,-0.010191
995,0.261571,0.240490,0.223076,0.205202,0.196953,0.173123,0.160291,0.153416,0.144251,0.119045,...,0.034263,0.040679,0.059468,0.064968,0.064510,0.073675,0.075967,0.070926,0.055344,0.041137


In [ ]:
x_test_dt.shape

(997, 3000)

In [ ]:
x_test_dt_rshp=np.array(x_test_dt).reshape(997,3000,1)

In [ ]:
x_test_dt_rshp

array([[[-0.04504252],
        [-0.06074903],
        [-0.03647533],
        ...,
        [ 0.08584504],
        [ 0.03824956],
        [ 0.01921137]],

       [[ 0.04709478],
        [ 0.05167761],
        [ 0.04801135],
        ...,
        [-0.09543128],
        [-0.12842767],
        [ 0.00080818]],

       [[ 0.03930397],
        [ 0.02968002],
        [ 0.02463891],
        ...,
        [-0.11559574],
        [-0.12476141],
        [-0.11467918]],

       ...,

       [[-0.00698263],
        [-0.00148324],
        [-0.00194152],
        ...,
        [-0.00698263],
        [-0.01110718],
        [-0.01019062]],

       [[ 0.2615713 ],
        [ 0.24049027],
        [ 0.22307551],
        ...,
        [ 0.07092551],
        [ 0.05534388],
        [ 0.0411371 ]],

       [[-0.0078992 ],
        [-0.00102495],
        [-0.00469122],
        ...,
        [-0.01019062],
        [-0.00377465],
        [-0.01248203]]])

In [ ]:
import numpy as np
np.random.seed(999)

import tensorflow
tensorflow.random.set_seed(999)

import random
#random.seed(999)

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-00fr1e3p
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-00fr1e3p
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=ddab0e25bd0cd9ab0ff4dbab897de9074c629a6f8fd964d5fa9a0e84f71101ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-f6kaeg40/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter

Fs = 100

In [ ]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)
    
    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)
    
    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)
    
    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

In [ ]:
import keras_tuner as kt

In [ ]:
optimizer=Adam(lr=1e-4)

Performing incremental tri-training again the keras way, without using train_on_batch() but just the fit() to get the get incremental checkpoints corresponding to the 3 models required for batching 

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
for layer in loaded_model_shuf1.layers[:23]:
  layer.trainable=False
#for layer in loaded_model_shuf1.layers[20:]:      #freezing all the layers of the model (non incremental feedback model)
 # layer.trainable=False

In [ ]:
for layer in loaded_model_shuf2.layers[:23]:
  layer.trainable=False
#for layer in loaded_model_shuf2.layers[20:]:      #freezing all the layers of the model (non incremental feedback model)
 # layer.trainable=False

In [ ]:
for layer in loaded_model_shuf3.layers[:23]:
  layer.trainable=False
#for layer in loaded_model_shuf3.layers[20:]:       #freezing all the layers of the model (non incremental feedback model)
#  layer.trainable=False

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(loaded_model_shuf1.summary())
print(loaded_model_shuf2.summary())
print(loaded_model_shuf3.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
tr_dt=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_train_sub.csv', header=None)

In [ ]:
tr_dt

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.001266,0.002183,0.001725,0.007224,0.008599,0.007224,-0.004233,-0.006066,-0.005608,-0.012482,...,0.041137,0.038387,0.035179,0.031055,0.030138,0.023264,0.012724,0.007224,0.002183,2
1,0.057177,0.074592,0.065426,0.065426,0.043887,0.029680,0.037929,0.037471,0.032888,0.017765,...,-0.022564,-0.022106,-0.025772,-0.009274,-0.043187,-0.033105,-0.030813,-0.028064,-0.040437,0
2,-0.093114,-0.161651,-0.194016,-0.213055,-0.218766,-0.220194,-0.221622,-0.214006,-0.184973,-0.155940,...,0.076326,0.081085,0.082513,0.077278,0.088225,0.088225,0.092508,0.085845,0.088225,0
3,-0.030813,-0.045020,-0.063351,-0.089015,-0.108721,-0.110555,-0.098181,-0.084432,-0.078017,-0.073892,...,0.028763,0.039304,0.039762,0.031513,0.022806,0.033805,0.026930,0.033805,0.040221,4
4,0.043009,0.041581,0.074422,0.055860,0.090129,0.075374,0.056812,0.064903,0.046817,0.075374,...,-0.119767,-0.109772,-0.073600,-0.039331,0.011120,0.073946,0.105359,0.190079,0.207214,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
3984,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
3985,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
3986,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,1


In [ ]:
x_sub_set=tr_dt.drop([3000],axis=1)
y_sub_dt=tr_dt[3000]

In [ ]:
loaded_model_shuf1_new=model_b(verbose=VBS)
loaded_model_shuf2_new=model_b(verbose=VBS)
loaded_model_shuf3_new=model_b(verbose=VBS)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf1_new.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2_new.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3_new.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
for layer in loaded_model_shuf1_new.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf1_new.layers[20:]:
  layer.trainable=True

In [ ]:
for layer in loaded_model_shuf2_new.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf2_new.layers[20:]:
  layer.trainable=True

In [ ]:
for layer in loaded_model_shuf3_new.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf3_new.layers[20:]:
  layer.trainable=True

In [ ]:
loaded_model_shuf1_new.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2_new.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3_new.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1_new.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf2_new.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3_new.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
checkpoint_path_model_shuf1 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf1/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf1 = os.path.dirname(checkpoint_path_model_shuf1)
cp_callback_model_shuf1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf1,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
checkpoint_path_model_shuf2 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf2/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf2 = os.path.dirname(checkpoint_path_model_shuf2)
cp_callback_model_shuf2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf2,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
checkpoint_path_model_shuf3 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf3/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf3 = os.path.dirname(checkpoint_path_model_shuf3)
cp_callback_model_shuf3 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf3,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
preds_shuf1=[]
preds_inc_shuf1=[]
probabilities_shuf1=[]
probabilities_inc_shuf1=[]


count_shuf1=0

preds_shuf2=[]
preds_inc_shuf2=[]
probabilities_shuf2=[]
probabilities_inc_shuf2=[]

count_shuf2=0

preds_shuf3=[]
preds_inc_shuf3=[]
probabilities_shuf3=[]
probabilities_inc_shuf3=[]

cnt_shuf3=0

m_vote_inc=[]

m_vote=[]

count_status1=0
count_status2=0
count_status3=0

for sple in np.array(x_sub_set[0:3988]):

  sple_rshp=sple.reshape(1,3000,1)



  p1_i=loaded_model_shuf1.predict(sple_rshp)

  p1_i_fin=np.argmax(p1_i,axis=1)

  preds_shuf1.append(np.asscalar(p1_i_fin))

  probabilities_shuf1.append(p1_i) 

  
  
  p1_i_inc=loaded_model_shuf1_new.predict(sple_rshp)

  p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

  
  #count_shuf1+=1

  p2_i=loaded_model_shuf2.predict(sple_rshp)

  p2_i_fin=np.argmax(p2_i,axis=1)

  preds_shuf2.append(np.asscalar(p2_i_fin))

  probabilities_shuf2.append(p2_i)

  
    
  p2_i_inc=loaded_model_shuf2_new.predict(sple_rshp)

  p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

  
  #count_shuf2+=1

  p3_i=loaded_model_shuf3.predict(sple_rshp)

  p3_i_fin=np.argmax(p3_i,axis=1)

  preds_shuf3.append(np.asscalar(p3_i_fin))

  probabilities_shuf3.append(p3_i)

  count_status3+=1

  
  p3_i_inc=loaded_model_shuf3_new.predict(sple_rshp)

  p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

  

  preds_inc_shuf1.append(np.asscalar(p1_i_inc_fin))

  probabilities_inc_shuf1.append(p1_i_inc)


  preds_inc_shuf2.append(np.asscalar(p2_i_inc_fin))

  probabilities_inc_shuf2.append(p2_i_inc)


  preds_inc_shuf3.append(np.asscalar(p3_i_inc_fin))

  probabilities_inc_shuf3.append(p3_i_inc)




  mode_i_inc=mode([np.asscalar(p1_i_inc_fin),np.asscalar(p2_i_inc_fin),np.asscalar(p3_i_inc_fin)])
  m_vote_i_inc=mode_i_inc[0][0]
  m_vote_inc.append(m_vote_i_inc)

  mode_i=mode([np.asscalar(p1_i_fin),np.asscalar(p2_i_fin),np.asscalar(p3_i_fin)])
  m_vote_i=mode_i[0][0]
  m_vote.append(m_vote_i)

  #count_shuf3+=1

  if (p1_i_inc_fin != p1_i_fin):
    p1_i_inc_fin=p1_i_fin

  if (p2_i_inc_fin != p2_i_fin):
    p2_i_inc_fin=p2_i_fin

  if (p3_i_inc_fin != p3_i_fin):
    p3_i_inc_fin=p3_i_fin

  
  #p1_i_inc_fin_vals=np.asscalar(p1_i_inc_fin)
  #p2_i_inc_fin_vals=np.asscalar(p2_i_inc_fin)
  #p3_i_inc_fin_vals=np.asscalar(p3_i_inc_fin)

  

  p1_i_inc_fin_=np_utils.to_categorical(np.asscalar(p1_i_inc_fin), num_classes=5)
  p2_i_inc_fin_=np_utils.to_categorical(np.asscalar(p2_i_inc_fin), num_classes=5)
  p3_i_inc_fin_=np_utils.to_categorical(np.asscalar(p3_i_inc_fin), num_classes=5)

  p1_i_inc_fin_.shape=(1,5)
  p2_i_inc_fin_.shape=(1,5)
  p3_i_inc_fin_.shape=(1,5)

  

  if (p2_i_inc_fin==p3_i_inc_fin):
    
    p1_i_inc_fin=p3_i_inc_fin

    p1_i_inc_fin_=np_utils.to_categorical(np.asscalar(p1_i_inc_fin), num_classes=5)

    p1_i_inc_fin_.shape=(1,5)

    #p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

    
        
    loaded_model_shuf1_new.fit(sple_rshp, p1_i_inc_fin_, callbacks=[cp_callback_model_shuf1])

    

        
   
        
    loaded_model_shuf2_new.fit(sple_rshp, p2_i_inc_fin_, callbacks=[cp_callback_model_shuf2])

   


   
        
    loaded_model_shuf3_new.fit(sple_rshp, p3_i_inc_fin_, callbacks=[cp_callback_model_shuf3])

    
     

  elif (p3_i_inc_fin==p1_i_inc_fin):
    
    p2_i_inc_fin=p1_i_inc_fin

    p2_i_inc_fin_=np_utils.to_categorical(np.asscalar(p2_i_inc_fin), num_classes=5)

    p2_i_inc_fin_.shape=(1,5)

    #p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

    
        
    loaded_model_shuf2_new.fit(sple_rshp, p2_i_inc_fin_, callbacks=[cp_callback_model_shuf2])

    
   
        
    loaded_model_shuf1_new.fit(sple_rshp, p1_i_inc_fin_, callbacks=[cp_callback_model_shuf1])

    

    
        
    loaded_model_shuf3_new.fit(sple_rshp, p3_i_inc_fin_, callbacks=[cp_callback_model_shuf3])

    
  elif (p2_i_inc_fin==p1_i_inc_fin):
    
    p3_i_inc_fin=p1_i_inc_fin

    p3_i_inc_fin_=np_utils.to_categorical(np.asscalar(p3_i_inc_fin), num_classes=5)

    p3_i_inc_fin_.shape=(1,5)

    #p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

    
        
    loaded_model_shuf3_new.fit(sple_rshp, p3_i_inc_fin_, callbacks=[cp_callback_model_shuf3])
   


    
        
    loaded_model_shuf1_new.fit(sple_rshp, p1_i_inc_fin_, callbacks=[cp_callback_model_shuf1])

       
    

    
        
    loaded_model_shuf2_new.fit(sple_rshp, p2_i_inc_fin_, callbacks=[cp_callback_model_shuf2])

   
  
 

  else:
 
      
   
        
    loaded_model_shuf1_new.fit(sple_rshp, p1_i_inc_fin_, callbacks=[cp_callback_model_shuf1])

    
    
        
    loaded_model_shuf2_new.fit(sple_rshp, p2_i_inc_fin_, callbacks=[cp_callback_model_shuf2])

    

    
    
        
    loaded_model_shuf3_new.fit(sple_rshp, p3_i_inc_fin_, callbacks=[cp_callback_model_shuf3])

     
  

Streaming output truncated to the last 5000 lines.
Epoch 1: saving model to /content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf2/weights-01-1.000.ckpt
1/1 [==============================] - ETA: 0s - loss: 0.0336 - accuracy: 1.0000
Epoch 1: saving model to /content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf3/weights-01-1.000.ckpt
1/1 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 1.0000
Epoch 1: saving model to /content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf1/weights-01-1.000.ckpt
1/1 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.0000
Epoch 1: saving model to /content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf2/weights-01-1.000.ckpt
1/1 [==============================] - ETA: 0s - loss: 0.0012 - accuracy: 1.0000
Epoch 1: saving model to /content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf3/weights-01-1.000.

In [ ]:
print(probabilities_inc_shuf1)
print(probabilities_inc_shuf2)
print(probabilities_inc_shuf3)

Streaming output truncated to the last 5000 lines.
        6.5150423e-05]], dtype=float32), array([[2.0847643e-04, 7.5607352e-02, 9.1878688e-01, 2.1101857e-04,
        5.1862490e-03]], dtype=float32), array([[9.7653085e-01, 1.1566868e-02, 1.5749244e-03, 6.8300636e-05,
        1.0259010e-02]], dtype=float32), array([[3.0450536e-05, 8.0272043e-03, 9.9014622e-01, 2.1591972e-04,
        1.5803060e-03]], dtype=float32), array([[9.9978703e-01, 1.0164139e-04, 1.7472523e-05, 7.4343072e-05,
        1.9538349e-05]], dtype=float32), array([[9.9860579e-01, 3.7647720e-04, 5.8348782e-05, 1.5384825e-04,
        8.0548052e-04]], dtype=float32), array([[9.9922729e-01, 5.5937079e-04, 1.4767647e-04, 4.1216590e-05,
        2.4455050e-05]], dtype=float32), array([[9.3509379e-04, 8.5799478e-02, 1.9980001e-01, 1.1270126e-04,
        7.1335274e-01]], dtype=float32), array([[0.0058457 , 0.12805547, 0.4639113 , 0.00077173, 0.40141582]],
      dtype=float32), array([[8.7938994e-02, 8.3660968e-03, 1.3636077e-03, 

In [ ]:
print(probabilities_shuf1)
print(probabilities_shuf2)
print(probabilities_shuf3)

Streaming output truncated to the last 5000 lines.
        9.5817784e-05]], dtype=float32), array([[2.4762307e-03, 4.5223597e-01, 5.2598280e-01, 9.4572941e-05,
        1.9210434e-02]], dtype=float32), array([[9.4597828e-01, 3.5465982e-02, 7.7975281e-03, 2.0232636e-05,
        1.0738041e-02]], dtype=float32), array([[4.3002504e-04, 6.9671832e-02, 9.1699493e-01, 1.5487228e-04,
        1.2748295e-02]], dtype=float32), array([[9.9979824e-01, 1.2613791e-04, 3.0876981e-05, 3.1778472e-05,
        1.2976499e-05]], dtype=float32), array([[9.9898344e-01, 7.4053864e-04, 6.1182276e-05, 3.3580927e-05,
        1.8126535e-04]], dtype=float32), array([[9.9871767e-01, 8.8223343e-04, 3.3602648e-04, 3.3268829e-05,
        3.0802163e-05]], dtype=float32), array([[7.8211157e-03, 2.9305220e-01, 3.2033706e-01, 6.7895715e-05,
        3.7872171e-01]], dtype=float32), array([[1.6589660e-02, 3.8284224e-01, 3.5850599e-01, 1.1055816e-04,
        2.4195158e-01]], dtype=float32), array([[3.9064375e-01, 3.2298006e-02

In [ ]:
#accuracy after incrementally training and predicting on x_train_sub[0:500](hold-out set) using loaded_model_shuf1_inc
from sklearn.metrics import accuracy_score
accuracy_score(y_sub_dt[0:3988], m_vote_inc)

0.7760782347041123

In [ ]:
#accuracy after incrementally training and predicting on x_train_sub[0:500](hold-out set) using loaded_model_shuf1_inc
from sklearn.metrics import accuracy_score
accuracy_score(y_sub_dt[0:3988], m_vote)

0.7921263791374122

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, m_vote_inc)
matrix.diagonal()/matrix.sum(axis=1)

array([0.91748142, 0.04955752, 0.89665653, 0.62980769])

In [ ]:
x_train_with_tri_trained_labels=x_sub_set

In [ ]:
x_train_with_tri_trained_labels['pseudo_labels']=m_vote_inc

In [ ]:
x_train_with_tri_trained_labels

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,pseudo_labels
0,0.001266,0.002183,0.001725,0.007224,0.008599,0.007224,-0.004233,-0.006066,-0.005608,-0.012482,...,0.041137,0.038387,0.035179,0.031055,0.030138,0.023264,0.012724,0.007224,0.002183,2
1,0.057177,0.074592,0.065426,0.065426,0.043887,0.029680,0.037929,0.037471,0.032888,0.017765,...,-0.022564,-0.022106,-0.025772,-0.009274,-0.043187,-0.033105,-0.030813,-0.028064,-0.040437,0
2,-0.093114,-0.161651,-0.194016,-0.213055,-0.218766,-0.220194,-0.221622,-0.214006,-0.184973,-0.155940,...,0.076326,0.081085,0.082513,0.077278,0.088225,0.088225,0.092508,0.085845,0.088225,0
3,-0.030813,-0.045020,-0.063351,-0.089015,-0.108721,-0.110555,-0.098181,-0.084432,-0.078017,-0.073892,...,0.028763,0.039304,0.039762,0.031513,0.022806,0.033805,0.026930,0.033805,0.040221,4
4,0.043009,0.041581,0.074422,0.055860,0.090129,0.075374,0.056812,0.064903,0.046817,0.075374,...,-0.119767,-0.109772,-0.073600,-0.039331,0.011120,0.073946,0.105359,0.190079,0.207214,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983,0.007788,0.000649,-0.011726,-0.019817,-0.029336,-0.035047,-0.036951,-0.037427,-0.037427,-0.038379,...,-0.013154,-0.018389,-0.024101,-0.041235,-0.047422,-0.060749,-0.054562,-0.045043,-0.038379,2
3984,-0.294326,-0.221001,-0.269121,-0.249873,-0.320907,-0.250331,-0.277370,-0.308533,-0.328239,-0.289743,...,-0.039062,-0.097264,-0.113304,-0.148134,-0.158216,-0.162341,-0.165549,-0.171506,-0.164632,0
3985,0.024639,0.031055,0.038846,0.029222,0.026472,0.018681,0.002183,-0.004691,0.012724,-0.001942,...,-0.008357,-0.006983,-0.010191,-0.020731,-0.021189,-0.013399,-0.007899,-0.015232,-0.024397,0
3986,-0.014315,-0.056019,-0.067934,-0.050978,-0.059685,-0.082141,-0.104597,-0.077558,-0.062435,-0.057394,...,-0.058310,-0.082599,-0.095890,-0.072517,-0.067934,-0.083058,-0.112388,-0.105972,-0.112846,2


In [ ]:
x_train_with_tri_trained_labels.to_csv(r'/content/gdrive/My Drive/physionet/ensemble/50_subs/x_train_with_semi_sup_tri-trained_labels_frozen_layers_keras_way_fit_method.csv', header=False, index=False)

KD with the 70 epochs teacher model,  80 epochs student model--> trial 1

KD with the 60 epochs teacher model, 70 epochs student model--> trial 2

KD with the 70 epochs teacher model, 80 epochs student model--> trial 3

In [ ]:
teacher_model=model_b(verbose=VBS) #60 epochs teacher model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path="/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/teacher_60_epochs_wts_trial_2.ckpt"
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 10s 25ms/step - loss: 1.4819 - accuracy: 0.7763


In [ ]:
preds=teacher_model.predict(x_test_dt_rshp)
preds_fin=np.argmax(preds,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_fin, digits=6))

              precision    recall  f1-score   support

           0   0.972696  0.901899  0.935961       632
           1   0.375000  0.118110  0.179641       127
           2   0.659574  0.885714  0.756098       175
           4   0.250000  0.539683  0.341709        63

    accuracy                       0.776329       997
   macro avg   0.564318  0.611351  0.553352       997
weighted avg   0.795932  0.776329  0.770498       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.90189873, 0.11811024, 0.88571429, 0.53968254])

In [ ]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)

    
    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
#student_model=get_model_cnn()
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
# Initialize and compile distiller
#execute this piece of code every single time, to get a student model(since the student model changes every time)
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=5
)

# Distill teacher to student
#distiller.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])
distiller.fit(x_dt_rshp, y_dt_, epochs=70)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/70
3773/3773 [==============================] - 158s 41ms/step - categorical_accuracy: 0.6676 - student_loss: 0.8278 - distillation_loss: 0.0018
Epoch 2/70
3773/3773 [==============================] - 157s 42ms/step - categorical_accuracy: 0.7475 - student_loss: 0.6095 - distillation_loss: 0.0013
Epoch 3/70
3773/3773 [==============================] - 156s 41ms/step - categorical_accuracy: 0.7710 - student_loss: 0.5492 - distillation_loss: 0.0012
Epoch 4/70
3773/3773 [==============================] - 155s 41ms/step - categorical_accuracy: 0.7814 - student_loss: 0.5250 - distillation_loss: 0.0012
Epoch 5/70
3773/3773 [==============================] - 154s 41ms/step - categorical_accuracy: 0.7906 - student_loss: 0.5070 - distillation_loss: 0.0011
Epoch 6/70
3773/3773 [==============================] - 153s 41ms/step - categorical_accuracy: 0.7959 - student_loss: 0.4921 - distillation_loss: 0.0011
Epoch 7/70
3773/3773 [==============================] - 155s 41ms/step - categoric

In [ ]:
accuracy_distiller = distiller.evaluate(x_test_dt_rshp, y_test_dt_)

32/32 [==============================] - 0s 12ms/step - categorical_accuracy: 0.7673 - student_loss: 0.9456


In [ ]:
#latest_student_60_epochs=tensorflow.train.latest_checkpoint(checkpoint_student_model_60_epochs_dir)

In [ ]:
#temp=5, alpha=0.9   -----> 84.85%

In [ ]:
preds = student_model.predict(x_test_dt_rshp)

In [ ]:
preds

array([[9.99999762e-01, 3.65453889e-09, 1.30385658e-09, 3.11090807e-14,
        2.17813934e-07],
       [9.72687423e-01, 1.64975263e-02, 3.87082109e-05, 3.94763215e-11,
        1.07763289e-02],
       [3.77360918e-03, 7.62168989e-02, 4.93681356e-02, 1.13652766e-06,
        8.70640218e-01],
       ...,
       [2.50406129e-06, 9.27175023e-03, 9.12512898e-01, 6.21117852e-06,
        7.82066062e-02],
       [4.95064817e-02, 4.63674396e-01, 1.01286866e-01, 1.49824723e-06,
        3.85530770e-01],
       [9.99961138e-01, 7.24433846e-07, 3.91630870e-08, 2.87912896e-13,
        3.80955207e-05]], dtype=float32)

In [ ]:
np.set_printoptions(precision=15,suppress=True)

In [ ]:
preds

array([[1.               , 0.000000000482258, 0.000000000042793,
        0.000000000000008, 0.00000000024492 ],
       [0.9972038        , 0.00091129675    , 0.00000068862875 ,
        0.               , 0.0018841863     ],
       [0.0015288414     , 0.09967436       , 0.18619165       ,
        0.000004438261   , 0.71260077       ],
       ...,
       [0.00000008119583 , 0.0021013983     , 0.97135115       ,
        0.000022759064   , 0.026524652      ],
       [0.039122015      , 0.3414286        , 0.09151908       ,
        0.000001108718   , 0.5279292        ],
       [0.9999695        , 0.000000019484792, 0.000000038681094,
        0.000000000000906, 0.0000305357     ]], dtype=float32)

In [ ]:
preds[0]

array([0.000000000000001, 0.00000002246737 , 0.99937314       ,
       0.00062689255    , 0.000000000000001], dtype=float32)

In [ ]:
preds[0].max()

0.99937314

In [ ]:
preds_student_model_fin=np.argmax(preds,axis=1)

In [ ]:
print(classification_report(y_test_dt, preds_student_model_fin, digits=6))

              precision    recall  f1-score   support

           0   0.968543  0.925633  0.946602       632
           1   0.209302  0.070866  0.105882       127
           2   0.695431  0.782857  0.736559       175
           4   0.222222  0.539683  0.314815        63

    accuracy                       0.767302       997
   macro avg   0.523875  0.579760  0.525965       997
weighted avg   0.776731  0.767302  0.762719       997



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test_dt, preds_student_model_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.92563291, 0.07086614, 0.78285714, 0.53968254])


# thresh < 0.8

# trial 1
accuracy of student on test before batching: 83.39%

class acc: array ([0.97463768, 0.02631579, 0.93125   , 0.78688525, 0.56962025])

    precision    recall  f1-score   support

           0       0.92      0.97      0.95       276
           1       0.25      0.03      0.05        38
           2       0.72      0.93      0.81       160
           3       0.84      0.79      0.81        61
           4       0.83      0.57      0.68        79

    accuracy                           0.83       614
    macro avg       0.71      0.66      0.66       614

    weighted avg       0.81      0.83      0.81       614


accuracy of student on test after batching: 85.016%

class acc: array ([0.96376812, 0.05263158, 0.95      , 0.75409836, 0.70886076])

    precision    recall  f1-score   support

           0       0.94      0.96      0.95       276
           1       0.40      0.05      0.09        38
           2       0.74      0.95      0.83       160
           3       0.88      0.75      0.81        61
           4       0.81      0.71      0.76        79

    accuracy                           0.85       614
    macro avg       0.76      0.69      0.69       614
    weighted avg       0.83      0.85      0.83       614


# thresh< 0.7

# trial 1

accuracy of student on text before batching: 83.88%

array ([0.96014493, 0.07894737, 0.93125   , 0.8852459 , 0.55696203])

    precision    recall  f1-score   support

           0       0.93      0.96      0.94       276
           1       0.33      0.08      0.13        38
           2       0.73      0.93      0.82       160
           3       0.86      0.89      0.87        61
           4       0.85      0.56      0.67        79

    accuracy                           0.84       614
    macro avg       0.74      0.68      0.69       614
    weighted avg       0.82      0.84      0.82       614

accuracy of student on text after batching:85.6677

array([0.96014493, 0.07894737, 0.95      , 0.85245902, 0.6835443 ])
    
    precision    recall  f1-score   support

           0       0.94      0.96      0.95       276
           1       0.50      0.08      0.14        38
           2       0.75      0.95      0.84       160
           3       0.87      0.85      0.86        61
           4       0.86      0.68      0.76        79

    accuracy                           0.86       614
    macro avg       0.78      0.71      0.71       614
    weighted avg       0.85      0.86      0.84       614


# New trial 2:

accuracy of student on test before batching: 83.55%

array ([0.956521739130435, 0.052631578947368, 0.9375           ,
       0.852459016393443, 0.569620253164557])

    precision    recall  f1-score   support

           0       0.93      0.96      0.94       276
           1       0.22      0.05      0.09        38
           2       0.73      0.94      0.82       160
           3       0.84      0.85      0.85        61
           4       0.85      0.57      0.68        79

    accuracy                           0.84       614
    macro avg       0.71      0.67      0.68       614
    weighted avg       0.81      0.84      0.81       614

accuracy of student on text after batching: 


# Trial 3:

accuracy of student on test before batching: 83.39%

array([0.96014493, 0.02631579, 0.95625   , 0.73770492, 0.60759494])

    precision    recall  f1-score   support

           0       0.92      0.96      0.94       276
           1       0.25      0.03      0.05        38
           2       0.72      0.96      0.82       160
           3       0.90      0.74      0.81        61
           4       0.81      0.61      0.70        79

    accuracy                           0.83       614
    macro avg       0.72      0.66      0.66       614
    weighted avg       0.81      0.83      0.81       614

accuracy of student on text after batching: 




In [ ]:
#not required since the student model changes every time, every time knowledge is distilled from he 40 epochs teacher model, a student model results from it
student_model.save("/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs/student_model_83.39.h5")
#loaded_model = load_model("network.h5")
#loss, accuracy = loaded_model.evaluate(test_data, test_targets)

In [ ]:
#not required, eason same as above
from keras.models import load_model
loaded_model = load_model("/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs/student_model_83.39.h5")
loss, accuracy = loaded_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 13ms/step - loss: 0.7764 - acc: 0.8339


In [ ]:
#not required, reason same as above
student_model.evaluate(x_test_dt_rshp, y_test_dt_)

20/20 [==============================] - 0s 2ms/step - loss: 0.8097 - acc: 0.8388


[0.8097179532051086, 0.8387622237205505]

Predictions and batching

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print(loaded_model_shuf1.summary())
print(loaded_model_shuf2.summary())
print(loaded_model_shuf3.summary())

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

In [ ]:
checkpoint_path_model_shuf1 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf1/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf1 = os.path.dirname(checkpoint_path_model_shuf1)
cp_callback_model_shuf1 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf1,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
checkpoint_path_model_shuf2 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf2/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf2 = os.path.dirname(checkpoint_path_model_shuf2)
cp_callback_model_shuf2 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf2,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
checkpoint_path_model_shuf3 = "/content/gdrive/My Drive/physionet/ensemble_inc_semi_sup_tri_train/model_shuf3/weights-{epoch:02d}-{accuracy:.3f}.ckpt"
checkpoint_dir_model_shuf3 = os.path.dirname(checkpoint_path_model_shuf3)
cp_callback_model_shuf3 = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path_model_shuf3,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=1)

In [ ]:
latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)
    
loaded_model_shuf1_inc = model_b(verbose=VBS)

loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

for layer in loaded_model_shuf1_inc.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf1_inc.layers[20:]:
  layer.trainable=True

loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])    

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

In [ ]:
latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

loaded_model_shuf2_inc = model_b(verbose=VBS)

loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

for layer in loaded_model_shuf2_inc.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf2_inc.layers[20:]:
  layer.trainable=True

loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

In [ ]:
latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

loaded_model_shuf3_inc = model_b(verbose=VBS)

loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

for layer in loaded_model_shuf3_inc.layers[:20]:
  layer.trainable=False
for layer in loaded_model_shuf3_inc.layers[20:]:
  layer.trainable=True

loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])   

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
x_test_dt

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.115570,-0.115570,-0.110424,-0.144301,-0.218060,-0.275952,-0.265661,-0.247221,-0.216345,-0.156309,...,0.083836,0.090269,0.088982,0.078262,0.066254,0.066683,0.071829,0.099274,0.128006,0.128006
1,-0.001501,0.005360,-0.018654,-0.033234,-0.028088,-0.041811,-0.048243,-0.054676,-0.057678,-0.021227,...,-0.026802,-0.002359,0.026373,0.025087,0.042669,0.040953,0.019512,0.001501,-0.019941,-0.025515
2,-0.217289,-0.202789,-0.189994,-0.138391,-0.105552,-0.050111,0.061199,0.103420,0.173788,0.111097,...,0.054375,0.042861,-0.011728,0.003625,0.010022,0.015140,-0.043714,-0.021110,-0.042008,-0.064184
3,-0.003199,-0.100008,-0.153744,-0.026228,-0.022816,-0.045420,-0.077832,0.006184,-0.007463,-0.088493,...,0.019831,-0.040302,-0.027508,-0.043287,0.052243,-0.037743,-0.082096,-0.086361,-0.076979,-0.159715
4,-0.015140,0.006610,0.049684,0.073993,0.068449,0.046272,0.063331,0.069302,-0.001919,-0.024096,...,0.131141,0.113229,0.070155,0.035184,0.021110,0.023669,-0.008316,-0.006610,-0.000213,-0.024949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.085082,0.089773,0.111097,0.126023,0.126023,0.118347,0.124744,0.136685,0.173788,0.195538,...,-0.049258,-0.036037,-0.003625,0.048831,0.037317,0.034331,0.002346,0.039449,0.043287,0.039875
610,0.003199,0.078258,0.191274,0.130288,0.138817,0.198524,0.181465,0.239465,0.233068,0.259936,...,-0.048405,0.042434,-0.030066,-0.125597,-0.037743,0.168244,-0.060773,-0.009596,-0.050537,-0.116214
611,0.112376,0.032625,0.057361,0.103420,0.048831,-0.019405,0.046699,0.028361,0.078685,0.061626,...,0.022816,0.018125,0.028361,0.088920,0.008743,-0.042008,0.085935,-0.185303,-0.135406,-0.201936
612,0.018225,0.008362,-0.019083,-0.067112,-0.049530,0.017368,0.037523,0.047386,0.026373,-0.008791,...,0.018654,-0.003216,0.004074,0.006218,-0.018225,-0.025944,-0.013508,0.011793,-0.036665,-0.057249


In [ ]:
checkpoint_teacher_model_i = "/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs/model_i/teacher_i_wts.ckpt" #teacher incremental model to be updated after every run

In [ ]:
checkpoint_teacher_model_i = "/content/gdrive/My Drive/physionet/ensemble/tri_training/60_epochs_keras_way/model_i/teacher_i_wts.ckpt" #teacher incremental model to be updated after every run

In [ ]:
checkpoint_teacher_model_i_dir = os.path.dirname(checkpoint_teacher_model_i)

In [ ]:
cp_callback_teacher_model_i = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_teacher_model_i,
    monitor='accuracy', 
    verbose=1, 
    save_weights_only=True,
    mode = 'max',
    period=60)

In [ ]:
#main code for tri training is in function called pro(), code for batching is in method called batching()

In [ ]:
def pro_new(df):

  

  #global preds_shuf1, preds_inc_shuf1, preds_shuf2, preds_inc_shuf2, preds_shuf3, preds_inc_shuf3, m_vote_inc, m_vote
  #global p2_i_inc_fin

  #mvote=[]

  m_vote_inc=[]
  m_vote=[]



  for sple in np.array(df):

    preds_shuf1=[]
    preds_inc_shuf1=[]

    preds_shuf2=[]
    preds_inc_shuf2=[]

    preds_shuf3=[]
    preds_inc_shuf3=[]

    #m_vote_inc=[]
    #m_vote=[]
    
    sple_rshp=sple.reshape(1,3000,1)

    p1_i=loaded_model_shuf1.predict(sple_rshp)
    
    p1_i_fin=np.argmax(p1_i,axis=1)
    
    preds_shuf1.append(p1_i_fin)


    
    p1_i_inc=loaded_model_shuf1_inc.predict_on_batch(sple_rshp)

    p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)


    p2_i=loaded_model_shuf2.predict(sple_rshp)

    p2_i_fin=np.argmax(p2_i,axis=1)

    preds_shuf2.append(p2_i_fin)


    
    p2_i_inc=loaded_model_shuf2_inc.predict_on_batch(sple_rshp)

    p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)


    p3_i=loaded_model_shuf3.predict(sple_rshp)

    p3_i_fin=np.argmax(p3_i,axis=1)

    preds_shuf3.append(p3_i_fin)


    
    
    p3_i_inc=loaded_model_shuf3_inc.predict_on_batch(sple_rshp)

    p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

    
    
    
    
    preds_inc_shuf1.append(np.asscalar(p1_i_inc_fin))

    #probabilities_inc_shuf1.append(p1_i_inc)


    preds_inc_shuf2.append(np.asscalar(p2_i_inc_fin))

    #probabilities_inc_shuf2.append(p2_i_inc)


    preds_inc_shuf3.append(np.asscalar(p3_i_inc_fin))

    #probabilities_inc_shuf3.append(p3_i_inc)


    mode_i_inc=mode([np.asscalar(p1_i_inc_fin),np.asscalar(p2_i_inc_fin),np.asscalar(p3_i_inc_fin)])
    m_vote_i_inc=mode_i_inc[0][0]
    m_vote_inc.append(m_vote_i_inc)

    mode_i=mode([np.asscalar(p1_i_fin),np.asscalar(p2_i_fin),np.asscalar(p3_i_fin)])
    m_vote_i=mode_i[0][0]
    m_vote.append(m_vote_i)



    if (p1_i_inc_fin != p1_i_fin):
      p1_i_inc_fin=p1_i_fin

    if (p2_i_inc_fin != p2_i_fin):
      p2_i_inc_fin=p2_i_fin

    if (p3_i_inc_fin != p3_i_fin):
      p3_i_inc_fin=p3_i_fin

    
    p1_i_inc_fin_=np_utils.to_categorical(np.asscalar(p1_i_inc_fin), num_classes=5)
    p2_i_inc_fin_=np_utils.to_categorical(np.asscalar(p2_i_inc_fin), num_classes=5)
    p3_i_inc_fin_=np_utils.to_categorical(np.asscalar(p3_i_inc_fin), num_classes=5)

    p1_i_inc_fin_.shape=(1,5)
    p2_i_inc_fin_.shape=(1,5)
    p3_i_inc_fin_.shape=(1,5)


    if (p2_i_inc_fin==p3_i_inc_fin):
      
      p1_i_inc_fin=p3_i_inc_fin

      p1_i_inc_fin_=np_utils.to_categorical(np.asscalar(p1_i_inc_fin), num_classes=5)

      p1_i_inc_fin_.shape=(1,5)

      #p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

    
        
      loaded_model_shuf1_inc.train_on_batch(sple_rshp, p1_i_inc_fin_)

    

        
   
        
      loaded_model_shuf2_inc.train_on_batch(sple_rshp, p2_i_inc_fin_)

   


   
        
      loaded_model_shuf3_inc.train_on_batch(sple_rshp, p3_i_inc_fin_)

    
     

    elif (p3_i_inc_fin==p1_i_inc_fin):
      
      p2_i_inc_fin=p1_i_inc_fin

      p2_i_inc_fin_=np_utils.to_categorical(np.asscalar(p2_i_inc_fin), num_classes=5)

      p2_i_inc_fin_.shape=(1,5)

      #p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

    
        
      loaded_model_shuf2_inc.train_on_batch(sple_rshp, p2_i_inc_fin_)

    
   
        
      loaded_model_shuf1_inc.train_on_batch(sple_rshp, p1_i_inc_fin_)

    

    
        
      loaded_model_shuf3_inc.train_on_batch(sple_rshp, p3_i_inc_fin_)

    
    elif (p2_i_inc_fin==p1_i_inc_fin):
      
      p3_i_inc_fin=p1_i_inc_fin

      p3_i_inc_fin_=np_utils.to_categorical(np.asscalar(p3_i_inc_fin), num_classes=5)

      p3_i_inc_fin_.shape=(1,5)

      #p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

    
        
      loaded_model_shuf3_inc.train_on_batch(sple_rshp, p3_i_inc_fin_)
   


    
        
      loaded_model_shuf1_inc.train_on_batch(sple_rshp, p1_i_inc_fin_)

       
    

    
        
      loaded_model_shuf2_inc.train_on_batch(sple_rshp, p2_i_inc_fin_)

   
  
 

    else:
      
      loaded_model_shuf1_inc.train_on_batch(sple_rshp, p1_i_inc_fin_)

    
    
        
      loaded_model_shuf2_inc.train_on_batch(sple_rshp, p2_i_inc_fin_)

       
    
        
      loaded_model_shuf3_inc.train_on_batch(sple_rshp, p3_i_inc_fin_) 
      

      
      
      
  return m_vote_inc

    

In [ ]:
round_cnt=0
return_counter=0
student_preds=[]
maj_votes_inc_cp=[]


def batching(x_test,student_mod):

  batch_df=pd.DataFrame()
  indices=[]
  indices_correct_preds=[]
  
  global return_counter, student_preds, round_cnt, maj_votes_inc_cp
  
  

  #for sple,rec in zip(np.array(x_test),x_test):
  for (sple,(ind,row)) in zip(np.array(x_test),x_test.iterrows()):
    
    sple_rshp=sple.reshape(1,3000,1)
    sple_df_rshp=sple.reshape(1,3000)
    sple_df_rshp_df=pd.DataFrame(sple_df_rshp)

    
      
    student_i_probs=student_mod.predict(sple_rshp)
    student_i_max_prob=student_i_probs.max()
    return_counter=return_counter+1

    if student_i_max_prob < 0.7:
      #batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))
      #batch_df=batch_df.append(pd.DataFrame(sple_df_rshp),ignore_index=False)
      batch_df=batch_df.append(row,ignore_index=False)

      #indices=indices.append(sple_df_rshp_df.index)

      indices.append(ind)
        
    else:
      
      student_i_fin=np.argmax(student_i_probs,axis=1)

      student_preds.append(np.asscalar(student_i_fin))

      indices_correct_preds.append(ind)



      #if not batch_df.empty:
        
        #round_cnt=round_cnt+1

      #print("the round number is",round_cnt)


  round_cnt=round_cnt+1

  original_stdout = sys.stdout 

  with open('/content/gdrive/My Drive/physionet/ensemble/batching_wrk/sub_473_outputs_0.7_trial_2_final.txt', 'a') as f:
    sys.stdout = f # Change the standard output to the file we created.
    #print('This message will be written to a file.')
    
    print("round ",round_cnt)
    print("indices of rows with confidence < 0.7 ", indices)
    print("length of batch df is ",len(batch_df))
    print("indices of rows with confidence > 0.7 ", indices_correct_preds)
    print("student predictions with confidence > 0.7",student_preds)
    
    sys.stdout = original_stdout # Reset the standard output to its original value     

  #round_cnt=round_cnt+1
  print("round ",round_cnt)
  print(indices)
    
  print("length of batch df is ",len(batch_df))



      
    
  #while batch_df.empty == False:

  if not batch_df.empty:
    
    len_df=len(batch_df)
    #proc_len=len_df+1

    maj_vote_inc=pro_new(batch_df[0:len_df])
    maj_votes_inc_cp=maj_vote_inc.copy()

    original_stdout1 = sys.stdout

    with open('/content/gdrive/My Drive/physionet/ensemble/batching_wrk/sub_473_outputs_0.7_trial_2_final.txt', 'a') as f:
      
      sys.stdout = f # Change the standard output to the file we created.

      print("predictions for batch df (confidence < 0.7)", maj_vote_inc)

      sys.stdout = original_stdout1 # Reset the standard output to its original value     

    #print("predictions for batch df", m_vote_inc)

    batch_df_cp=batch_df.copy()
    batch_df_cp['sleep_stage_voted']=maj_vote_inc
    batch_df_x=batch_df.copy()

    batch_df_x_rshp=np.array(batch_df_x).reshape(len(batch_df_x),3000,1)

    batch_df_y_=np_utils.to_categorical(maj_vote_inc)

    teacher_model_i=model_b(verbose=VBS) #50 epochs teacher model

    teacher_model_i.load_weights(teacher_checkpt_path)
    teacher_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    teacher_model_i.fit(batch_df_x_rshp, batch_df_y_, epochs=70, callbacks=[cp_callback_teacher_model_i]) #write code in the previous cell to checkpoint every teacher model that is trained to every chunk of the batch_df data frame
    #fit the teacher model according to the batch df chunk and check point it

    student_model_i=get_model_1d_cnn()
    student_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



    # Initialize and compile distiller
    distiller_i = Distiller(student=student_model_i, teacher=teacher_model_i)
    distiller_i.compile(optimizer=keras.optimizers.Adam(),
                        
                        metrics=[keras.metrics.CategoricalAccuracy()],
                        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True), #perform kd on the student model
                        distillation_loss_fn=keras.losses.KLDivergence(),
                        alpha=0.9,temperature=5)

    # Distill teacher to student
    #distiller.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])
    distiller_i.fit(batch_df_x_rshp, batch_df_y_, epochs=80)

    print("predictions for batch df", maj_vote_inc)

    #return batching(batch_df_x,student_model_i) 
    batching(batch_df_x,student_model_i)




  return maj_votes_inc_cp  #call the batching() by passing batch_df_x

       
       #use the student model to perform preds on the batch df one at a time(go batck to the first line of the function)
       #check confidence, generate batch df....and continue until batch df is empty


In [ ]:
m=batching(x_test_dt[0:997],student_model)

round  1
[3, 11, 12, 13, 16, 18, 23, 26, 28, 30, 31, 32, 35, 37, 38, 40, 42, 53, 54, 55, 60, 61, 64, 70, 77, 82, 92, 93, 101, 105, 106, 117, 118, 119, 120, 125, 130, 137, 140, 142, 149, 154, 155, 158, 160, 172, 174, 182, 186, 187, 191, 192, 195, 201, 207, 209, 211, 217, 219, 227, 237, 246, 251, 252, 254, 256, 264, 276, 277, 284, 286, 292, 294, 299, 304, 312, 316, 318, 321, 322, 324, 327, 329, 331, 332, 336, 341, 345, 348, 351, 357, 358, 360, 368, 369, 374, 376, 377, 384, 386, 390, 392, 394, 403, 405, 414, 417, 418, 419, 421, 428, 431, 439, 445, 450, 452, 456, 458, 462, 464, 467, 468, 474, 475, 479, 480, 487, 497, 502, 504, 517, 518, 519, 521, 525, 528, 529, 541, 549, 550, 553, 557, 563, 564, 566, 569, 572, 573, 584, 591, 592, 594, 595, 596, 614, 617, 621, 626, 627, 632, 634, 637, 647, 659, 665, 666, 668, 673, 674, 679, 683, 692, 696, 698, 701, 704, 708, 711, 714, 715, 717, 720, 722, 724, 731, 749, 751, 755, 756, 758, 763, 764, 769, 775, 778, 782, 783, 789, 800, 803, 812, 822, 825, 839,

In [ ]:
print(m)

[2, 2, 1, 4, 1, 4]


<0.8

In [ ]:
preds_greater_1="aarray([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([3]), array([2]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([2]), array([3]), array([2]), array([2]), array([3]), array([2]), array([2]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([2]), array([4]), array([2]), array([2]), array([0]), array([3]), array([2]), array([0]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([3]), array([0]), array([2]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([4]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([2]), array([0]), array([3]), array([0]), array([0]), array([3]), array([0]), array([4]), array([0]), array([3]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([2]), array([2]), array([4]), array([2]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([3]), array([4]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([2]), array([3]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([2]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([4]), array([3]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([4]), array([0]), array([2]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([0]), array([2]), array([2]), array([0]), array([4]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([0]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([3]), array([2]), array([3]), array([2]), array([4]), array([3]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([3]), array([0]), array([2]), array([4]), array([2]), array([2]), array([2]), array([0]), array([4]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0])"

In [ ]:
chunks_greater_1 = preds_greater_1.split(',')

In [ ]:
print(chunks_greater_1)

['aarray([2])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([3])', ' array([0])', ' array([0])', ' array([2])', ' array([2])', ' array([2])', ' array([0])', ' array([0])', ' array([4])', ' array([0])', ' array([0])', ' array([0])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([4])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([2])', ' array([0])', ' array([4])', ' array([2])', ' array([

In [ ]:
fin_preds_greater_1 = [chunk_greater_1[8] for chunk_greater_1 in chunks_greater_1]

In [ ]:
print(fin_preds_greater_1)

['2', '2', '0', '0', '2', '0', '0', '2', '0', '0', '3', '0', '0', '2', '2', '2', '0', '0', '4', '0', '0', '0', '3', '0', '0', '0', '0', '2', '0', '0', '2', '3', '0', '0', '0', '2', '0', '4', '2', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '2', '3', '0', '0', '0', '0', '2', '2', '0', '4', '2', '3', '2', '0', '3', '0', '2', '2', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '2', '0', '2', '0', '0', '0', '0', '0', '3', '2', '3', '2', '2', '3', '2', '2', '3', '0', '3', '0', '0', '0', '3', '2', '3', '0', '0', '0', '0', '3', '0', '0', '0', '3', '0', '0', '2', '2', '0', '2', '0', '0', '2', '2', '0', '2', '2', '0', '0', '2', '0', '0', '4', '0', '0', '0', '0', '0', '2', '0', '4', '2', '2', '4', '2', '2', '0', '3', '2', '0', '0', '0', '2', '3', '2', '2', '0', '3', '0', '2', '0', '0', '3', '0', '0', '2', '2', '0', '0', '2', '2', '0', '0', '4', '0', '2', '0', '0', '2', '0', '2', '0', '0', '0', '3', '0', '2', '2', '2', '0', '3', '0', '0', '3', '0', '4', '0',

In [ ]:
fin_preds_greater_1_int = list(map(int, fin_preds_greater_1))

In [ ]:
print(fin_preds_greater_1_int)

[2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 0, 0, 2, 2, 2, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 2, 2, 0, 4, 2, 3, 2, 0, 3, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 3, 2, 3, 2, 2, 3, 2, 2, 3, 0, 3, 0, 0, 0, 3, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 4, 2, 2, 4, 2, 2, 0, 3, 2, 0, 0, 0, 2, 3, 2, 2, 0, 3, 0, 2, 0, 0, 3, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 4, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0, 0, 3, 0, 4, 0, 3, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 0, 2, 2, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 4, 2, 2, 4, 2, 0, 0, 3, 0, 4, 2, 2, 0, 0, 2, 0, 0, 0, 4, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 4, 2, 2, 3, 0, 3, 0, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 2, 2, 3, 0, 2, 0, 2, 0, 2, 2, 4, 2, 0, 0, 2, 

In [ ]:
inds_greater_1=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 29, 30, 32, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 231, 232, 234, 235, 236, 237, 238, 240, 241, 242, 245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 257, 258, 260, 261, 262, 263, 264, 266, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 287, 288, 289, 290, 291, 292, 293, 294, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 335, 337, 338, 340, 341, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 388, 389, 390, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 435, 436, 437, 438, 440, 441, 442, 443, 444, 445, 446, 448, 449, 450, 451, 452, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 475, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 515, 516, 517, 518, 519, 520, 521, 522, 523, 525, 526, 527, 528, 531, 533, 534, 535, 536, 537, 538, 539, 540, 541, 544, 545, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 558, 559, 560, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 579, 580, 581, 582, 583, 584, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 597, 598, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 613]

In [ ]:
len(inds_greater_1)

536

In [ ]:
len(fin_preds_greater_1_int)

536

In [ ]:
preds_lesser_1="aarray([4]), array([1]), array([2]), array([4]), array([2]), array([3]), array([4]), array([2]), array([0]), array([4]), array([2]), array([4]), array([4]), array([0]), array([4]), array([4]), array([3]), array([2]), array([4]), array([4]), array([4]), array([2]), array([2]), array([1]), array([2]), array([4]), array([4]), array([2]), array([4]), array([4]), array([2]), array([2]), array([4]), array([4]), array([2]), array([4]), array([4]), array([2]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([3]), array([2]), array([1]), array([2]), array([3]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([3]), array([4]), array([4]), array([3]), array([2]), array([4]), array([4]), array([4]), array([3]), array([4]), array([2]), array([3]), array([1])"

In [ ]:
chunks_lesser_1 = preds_lesser_1.split(',')

In [ ]:
print(chunks_lesser_1)

['aarray([4])', ' array([1])', ' array([2])', ' array([4])', ' array([2])', ' array([3])', ' array([4])', ' array([2])', ' array([0])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([0])', ' array([4])', ' array([4])', ' array([3])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([1])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([3])', ' array([2])', ' array([1])', ' array([2])', ' array([3])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([3])', ' array([

In [ ]:
fin_preds_lesser_1 = [chunk_lesser_1[8] for chunk_lesser_1 in chunks_lesser_1]

In [ ]:
print(fin_preds_lesser_1)

['4', '1', '2', '4', '2', '3', '4', '2', '0', '4', '2', '4', '4', '0', '4', '4', '3', '2', '4', '4', '4', '2', '2', '1', '2', '4', '4', '2', '4', '4', '2', '2', '4', '4', '2', '4', '4', '2', '2', '4', '4', '4', '4', '4', '4', '4', '3', '2', '1', '2', '3', '4', '4', '4', '4', '4', '4', '4', '4', '2', '4', '4', '4', '4', '4', '3', '4', '4', '3', '2', '4', '4', '4', '3', '4', '2', '3', '1']


In [ ]:
fin_preds_lesser_1_int = list(map(int, fin_preds_lesser_1))

In [ ]:
print(fin_preds_lesser_1_int)

[4, 1, 2, 4, 2, 3, 4, 2, 0, 4, 2, 4, 4, 0, 4, 4, 3, 2, 4, 4, 4, 2, 2, 1, 2, 4, 4, 2, 4, 4, 2, 2, 4, 4, 2, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 3, 4, 4, 3, 2, 4, 4, 4, 3, 4, 2, 3, 1]


In [ ]:
inds_lesser_1=[8, 11, 23, 25, 28, 31, 33, 38, 40, 56, 68, 81, 91, 95, 105, 113, 118, 120, 132, 143, 151, 174, 182, 194, 204, 226, 230, 233, 239, 243, 244, 253, 256, 259, 265, 267, 268, 286, 295, 306, 331, 332, 333, 334, 336, 339, 342, 352, 372, 387, 391, 392, 418, 434, 439, 447, 453, 454, 474, 476, 491, 514, 524, 529, 530, 532, 542, 543, 546, 557, 561, 562, 563, 578, 585, 596, 599, 612]

In [ ]:
preds_lesser_2="aarray([2]), array([2]), array([2]), array([3]), array([4]), array([2]), array([0]), array([2]), array([4]), array([1]), array([4]), array([2]), array([3]), array([4]), array([4]), array([4]), array([2]), array([2]), array([2]), array([2]), array([4]), array([2]), array([1]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([4]), array([2]), array([2]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([3]), array([4]), array([2]), array([4]), array([4]), array([4]), array([2]), array([2]), array([2]), array([1])"

In [ ]:
chunks_lesser_2 = preds_lesser_2.split(',')

In [ ]:
print(chunks_lesser_2)

['aarray([2])', ' array([2])', ' array([2])', ' array([3])', ' array([4])', ' array([2])', ' array([0])', ' array([2])', ' array([4])', ' array([1])', ' array([4])', ' array([2])', ' array([3])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([2])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([3])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([1])']


In [ ]:
fin_preds_lesser_2 = [chunk_lesser_2[8] for chunk_lesser_2 in chunks_lesser_2]

In [ ]:
fin_preds_lesser_2_int = list(map(int, fin_preds_lesser_2))

In [ ]:
print(fin_preds_lesser_2_int)

[2, 2, 2, 3, 4, 2, 0, 2, 4, 1, 4, 2, 3, 4, 4, 4, 2, 2, 2, 2, 4, 2, 1, 4, 4, 4, 4, 4, 2, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 3, 4, 2, 4, 4, 4, 2, 2, 2, 1]


In [ ]:
inds_lesser_2=[11, 23, 28, 31, 33, 38, 40, 68, 81, 95, 105, 118, 120, 143, 151, 174, 182, 244, 253, 265, 267, 286, 295, 332, 333, 334, 336, 339, 342, 372, 387, 391, 434, 439, 454, 491, 514, 524, 529, 530, 532, 542, 546, 561, 562, 563, 578, 596, 599, 612]

In [ ]:
len(inds_lesser_2)

50

In [ ]:
len(fin_preds_lesser_2_int)

50

In [ ]:
preds_lesser_3="aarray([1]), array([2]), array([2]), array([4]), array([2]), array([4]), array([2]), array([4]), array([2]), array([2]), array([2]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([1])"

In [ ]:
chunks_lesser_3 = preds_lesser_3.split(',')

In [ ]:
print(chunks_lesser_3)

['aarray([1])', ' array([2])', ' array([2])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([1])']


In [ ]:
fin_preds_lesser_3 = [chunk_lesser_3[8] for chunk_lesser_3 in chunks_lesser_3]

In [ ]:
fin_preds_lesser_3_int = list(map(int, fin_preds_lesser_3))

In [ ]:
print(fin_preds_lesser_3_int)

[1, 2, 2, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 1]


In [ ]:
inds_lesser_3=[11, 28, 143, 151, 182, 244, 265, 267, 295, 336, 387, 454, 491, 524, 529, 542, 561, 562, 612]

In [ ]:
len(inds_lesser_3)

19

In [ ]:
len(fin_preds_lesser_3_int)

19

In [ ]:
preds_lesser_4="aarray([1]), array([2]), array([4]), array([4]), array([2]), array([2]), array([2]), array([4]), array([2]), array([4]), array([1]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([1])"

In [ ]:
chunks_lesser_4 = preds_lesser_4.split(',')

In [ ]:
print(chunks_lesser_4)

['aarray([1])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([2])', ' array([4])', ' array([1])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([1])']


In [ ]:
fin_preds_lesser_4 = [chunk_lesser_4[8] for chunk_lesser_4 in chunks_lesser_4]

In [ ]:
fin_preds_lesser_4_int = list(map(int, fin_preds_lesser_4))

In [ ]:
print(fin_preds_lesser_4_int)

[1, 2, 4, 4, 2, 2, 2, 4, 2, 4, 1, 2, 4, 4, 4, 4, 4, 4, 1]


In [ ]:
inds_lesser_4=[11, 28, 143, 151, 182, 244, 265, 267, 295, 336, 387, 454, 491, 524, 529, 542, 561, 562, 612]

In [ ]:
len(inds_lesser_4)

19

In [ ]:
len(fin_preds_lesser_4_int)

19

In [ ]:
preds_lesser_5="aarray([1]), array([2]), array([4]), array([4]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([4]), array([4]), array([4]), array([1]), array([2]), array([4]), array([4]), array([1])"

In [ ]:
chunks_lesser_5 = preds_lesser_5.split(',')

In [ ]:
print(chunks_lesser_5)

['aarray([1])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([1])', ' array([2])', ' array([4])', ' array([4])', ' array([1])']


In [ ]:
fin_preds_lesser_5 = [chunk_lesser_5[8] for chunk_lesser_5 in chunks_lesser_5]

In [ ]:
fin_preds_lesser_5_int = list(map(int, fin_preds_lesser_5))

In [ ]:
print(fin_preds_lesser_5_int)

[1, 2, 4, 4, 2, 2, 2, 2, 2, 2, 4, 4, 4, 1, 2, 4, 4, 1]


In [ ]:
inds_lesser_5=[11, 28, 143, 151, 182, 244, 265, 295, 336, 387, 454, 491, 524, 529, 542, 561, 562, 612]

In [ ]:
len(inds_lesser_5)

18

In [ ]:
len(fin_preds_lesser_5_int)

18

In [ ]:
#Final predictions for rows < 0.7
fin_inds=[8, 11, 23, 25, 28, 31, 33, 38, 40, 56, 68, 81, 91, 95, 105, 113, 118, 120, 132, 143, 151, 174, 182, 194, 204, 226, 230, 233, 239, 243, 244, 253, 256, 259, 265, 267, 268, 286, 295, 306, 331, 332, 333, 334, 336, 339, 342, 352, 372, 387, 391, 392, 418, 434, 439, 447, 453, 454, 474, 476, 491, 514, 524, 529, 530, 532, 542, 543, 546, 557, 561, 562, 563, 578, 585, 596, 599, 612]

In [ ]:
fin_preds=[4,
1,
2,
4,
2,
3,
4,
2,
0,
4,
2,
4,
4,
1,
4,
4,
2,
3,
4,
4,
4,
4,
2,
1,
2,
4,
4,
2,
4,
4,
2,
2,
4,
4,
2,
4,
4,
2,
2,
4,
4,
4,
4,
4,
2,
4,
2,
2,
4,
2,
2,
4,
4,
2,
4,
4,
4,
4,
4,
2,
4,
4,
4,
1,
4,
3,
2,
4,
2,
2,
4,
4,
4,
2,
4,
2,
2,
1
]

In [ ]:
all_indices=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 29, 30, 32, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 231, 232, 234, 235, 236, 237, 238, 240, 241, 242, 245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 257, 258, 260, 261, 262, 263, 264, 266, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 287, 288, 289, 290, 291, 292, 293, 294, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 335, 337, 338, 340, 341, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 388, 389, 390, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 435, 436, 437, 438, 440, 441, 442, 443, 444, 445, 446, 448, 449, 450, 451, 452, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 475, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 515, 516, 517, 518, 519, 520, 521, 522, 523, 525, 526, 527, 528, 531, 533, 534, 535, 536, 537, 538, 539, 540, 541, 544, 545, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 558, 559, 560, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 579, 580, 581, 582, 583, 584, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 597, 598, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 613,
            8, 11, 23, 25, 28, 31, 33, 38, 40, 56, 68, 81, 91, 95, 105, 113, 118, 120, 132, 143, 151, 174, 182, 194, 204, 226, 230, 233, 239, 243, 244, 253, 256, 259, 265, 267, 268, 286, 295, 306, 331, 332, 333, 334, 336, 339, 342, 352, 372, 387, 391, 392, 418, 434, 439, 447, 453, 454, 474, 476, 491, 514, 524, 529, 530, 532, 542, 543, 546, 557, 561, 562, 563, 578, 585, 596, 599, 612]

In [ ]:
all_preds=[2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 0, 0, 2, 2, 2, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 3, 0, 0, 0, 0, 2, 2, 0, 4, 2, 3, 2, 0, 3, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 3, 2, 3, 2, 2, 3, 2, 2, 3, 0, 3, 0, 0, 0, 3, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 4, 2, 2, 4, 2, 2, 0, 3, 2, 0, 0, 0, 2, 3, 2, 2, 0, 3, 0, 2, 0, 0, 3, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 4, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0, 0, 3, 0, 4, 0, 3, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 0, 2, 2, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 4, 2, 2, 4, 2, 0, 0, 3, 0, 4, 2, 2, 0, 0, 2, 0, 0, 0, 4, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 4, 2, 2, 3, 0, 3, 0, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 2, 2, 3, 0, 2, 0, 2, 0, 2, 2, 4, 2, 0, 0, 2, 0, 4, 0, 4, 3, 2, 0, 2, 2, 0, 0, 2, 2, 3, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 4, 0, 0, 0, 0, 2, 2, 4, 0, 2, 3, 0, 3, 0, 0, 0, 0, 0, 3, 0, 4, 2, 0, 2, 2, 0, 4, 0, 0, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 3, 0, 2, 2, 3, 2, 0, 0, 2, 0, 0, 4, 0, 2, 0, 0, 0, 3, 0, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 3, 3, 2, 3, 2, 4, 3, 0, 0, 0, 2, 2, 0, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 3, 2, 3, 0, 2, 4, 2, 2, 2, 0, 4, 0, 2, 0, 2, 2, 0, 0, 0, 0, 4, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0,
           4,
1,
2,
4,
2,
3,
4,
2,
0,
4,
2,
4,
4,
1,
4,
4,
2,
3,
4,
4,
4,
4,
2,
1,
2,
4,
4,
2,
4,
4,
2,
2,
4,
4,
2,
4,
4,
2,
2,
4,
4,
4,
4,
4,
2,
4,
2,
2,
4,
2,
2,
4,
4,
2,
4,
4,
4,
4,
4,
2,
4,
4,
4,
1,
4,
3,
2,
4,
2,
2,
4,
4,
4,
2,
4,
2,
2,
1
]

In [ ]:
y_test_dt

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
all_preds_ser=pd.Series(all_preds)

In [ ]:
all_preds_ser

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    4
611    2
612    2
613    1
Length: 614, dtype: int64

In [ ]:
y_test_cp=y_test_dt.copy()

In [ ]:
y_test_cp

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
new_y_test=y_test_cp.reindex(all_indices) #reordering y_test_dt according to all_preds_ser 's order

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(new_y_test, all_preds_ser)

0.8501628664495114

In [ ]:
print(classification_report(new_y_test, all_preds_ser))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       276
           1       0.40      0.05      0.09        38
           2       0.74      0.95      0.83       160
           3       0.88      0.75      0.81        61
           4       0.81      0.71      0.76        79

    accuracy                           0.85       614
   macro avg       0.76      0.69      0.69       614
weighted avg       0.83      0.85      0.83       614



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(new_y_test, all_preds_ser)
matrix.diagonal()/matrix.sum(axis=1)

array([0.96376812, 0.05263158, 0.95      , 0.75409836, 0.70886076])

In [ ]:
preds_greater_1="aarray([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([4]), array([0]), array([0]), array([1]), array([3]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([3]), array([2]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([1]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([3]), array([2]), array([3]), array([2]), array([2]), array([3]), array([2]), array([4]), array([2]), array([3]), array([0]), array([3]), array([3]), array([0]), array([0]), array([3]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([0]), array([4]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([4]), array([2]), array([2]), array([0]), array([3]), array([2]), array([0]), array([4]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([3]), array([0]), array([2]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([4]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([2]), array([0]), array([3]), array([0]), array([0]), array([3]), array([0]), array([4]), array([0]), array([3]), array([4]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([4]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([4]), array([2]), array([4]), array([0]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([3]), array([2]), array([4]), array([3]), array([2]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([3]), array([4]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([4]), array([4]), array([2]), array([4]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([2]), array([3]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([2]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([3]), array([2]), array([0]), array([2]), array([3]), array([4]), array([2]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([0]), array([2]), array([2]), array([0]), array([4]), array([4]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([4]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([0]), array([3]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([2]), array([2]), array([4]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([3]), array([2]), array([2]), array([3]), array([2]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([3]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([4]), array([4]), array([0]), array([2]), array([2]), array([3]), array([2]), array([3]), array([0]), array([2]), array([4]), array([2]), array([2]), array([2]), array([0]), array([4]), array([2]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([2]), array([3]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0])"

In [ ]:
chunks_greater_1 = preds_greater_1.split(',')

In [ ]:
print(chunks_greater_1)

['aarray([2])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([4])', ' array([0])', ' array([0])', ' array([1])', ' array([3])', ' array([0])', ' array([0])', ' array([2])', ' array([2])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([3])', ' array([0])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([3])', ' array([0])', ' array([4])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([4])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([0])', ' array([0])', ' array([2])', ' array([3])', ' array([0])', ' array([0])', ' array([0])', ' array([0])', ' array([2])', ' array([2])', ' array([0])', ' array([

In [ ]:
fin_preds_greater_1 = [chunk_greater_1[8] for chunk_greater_1 in chunks_greater_1]

In [ ]:
print(fin_preds_greater_1)

['2', '2', '0', '0', '2', '0', '0', '2', '4', '0', '0', '1', '3', '0', '0', '2', '2', '2', '0', '0', '0', '3', '0', '3', '0', '0', '0', '3', '0', '4', '2', '0', '0', '2', '3', '0', '0', '0', '2', '0', '4', '2', '0', '0', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '2', '3', '0', '0', '0', '0', '2', '2', '0', '3', '2', '0', '3', '0', '2', '2', '0', '0', '0', '0', '2', '0', '0', '0', '1', '0', '2', '0', '2', '0', '0', '0', '0', '0', '2', '3', '2', '3', '2', '2', '3', '2', '4', '2', '3', '0', '3', '3', '0', '0', '3', '2', '3', '0', '0', '0', '0', '3', '0', '0', '0', '3', '0', '0', '2', '2', '0', '2', '0', '4', '0', '2', '2', '0', '2', '2', '0', '0', '2', '0', '0', '0', '0', '0', '0', '0', '2', '0', '2', '4', '2', '2', '0', '3', '2', '0', '4', '0', '0', '2', '3', '2', '2', '0', '3', '0', '2', '0', '0', '3', '0', '0', '2', '0', '0', '2', '2', '0', '0', '4', '0', '2', '0', '0', '2', '0', '2', '0', '0', '0', '3', '0', '2', '2', '2', '0', '3', '0', '0', '3', '0', '4', '0',

In [ ]:
fin_preds_greater_1_int = list(map(int, fin_preds_greater_1))

In [ ]:
print(fin_preds_greater_1_int)

[2, 2, 0, 0, 2, 0, 0, 2, 4, 0, 0, 1, 3, 0, 0, 2, 2, 2, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 4, 2, 0, 0, 2, 3, 0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0, 0, 2, 2, 0, 3, 2, 0, 3, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 2, 3, 2, 4, 2, 3, 0, 3, 3, 0, 0, 3, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 2, 2, 0, 2, 0, 4, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 2, 0, 3, 2, 0, 4, 0, 0, 2, 3, 2, 2, 0, 3, 0, 2, 0, 0, 3, 0, 0, 2, 0, 0, 2, 2, 0, 0, 4, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0, 0, 3, 0, 4, 0, 3, 4, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 4, 2, 2, 2, 0, 0, 0, 0, 0, 2, 4, 0, 3, 0, 0, 0, 0, 2, 4, 2, 4, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 4, 3, 2, 4, 3, 2, 0, 0, 3, 0, 4, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 0, 0, 4, 4, 2, 4, 2, 2, 0, 2, 2, 0, 4, 2, 2, 3, 0, 3, 0, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 2, 2, 3, 0, 2, 0, 2, 0, 2, 2, 

In [ ]:
inds_greater_1=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 234, 235, 236, 237, 238, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 251, 254, 255, 256, 257, 258, 260, 261, 262, 263, 264, 265, 266, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294, 296, 297, 298, 299, 300, 301, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 332, 334, 335, 336, 337, 338, 340, 341, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 356, 357, 358, 359, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 386, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 419, 420, 422, 423, 424, 426, 427, 428, 429, 431, 432, 433, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 448, 449, 450, 451, 452, 453, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 494, 496, 497, 498, 499, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 525, 526, 527, 528, 529, 531, 533, 535, 536, 537, 538, 540, 541, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 558, 559, 560, 561, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613]

In [ ]:
len(inds_greater_1)

548

In [ ]:
len(fin_preds_greater_1_int)

548

In [ ]:
preds_lesser_1="aarray([4]), array([0]), array([4]), array([2]), array([2]), array([0]), array([4]), array([4]), array([2]), array([1]), array([2]), array([4]), array([0]), array([4]), array([2]), array([2]), array([2]), array([4]), array([4]), array([4]), array([2]), array([2]), array([4]), array([1]), array([2]), array([4]), array([2]), array([1]), array([4]), array([4]), array([2]), array([4]), array([4]), array([2]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([2]), array([2]), array([4]), array([4]), array([4]), array([1]), array([2]), array([1]), array([4]), array([4]), array([4]), array([4]), array([4]), array([3]), array([1]), array([4]), array([4]), array([2]), array([4]), array([2]), array([4]), array([2]), array([4]), array([1]), array([2])"

In [ ]:
chunks_lesser_1 = preds_lesser_1.split(',')

In [ ]:
print(chunks_lesser_1)

['aarray([4])', ' array([0])', ' array([4])', ' array([2])', ' array([2])', ' array([0])', ' array([4])', ' array([4])', ' array([2])', ' array([1])', ' array([2])', ' array([4])', ' array([0])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([1])', ' array([2])', ' array([4])', ' array([2])', ' array([1])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([1])', ' array([2])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([3])', ' array([1])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([4])', ' array([1])', ' array([2])']


In [ ]:
fin_preds_lesser_1 = [chunk_lesser_1[8] for chunk_lesser_1 in chunks_lesser_1]

In [ ]:
print(fin_preds_lesser_1)

['4', '0', '4', '2', '2', '0', '4', '4', '2', '1', '2', '4', '0', '4', '2', '2', '2', '4', '4', '4', '2', '2', '4', '1', '2', '4', '2', '1', '4', '4', '2', '4', '4', '2', '2', '4', '4', '4', '4', '4', '2', '2', '2', '4', '4', '4', '1', '2', '1', '4', '4', '4', '4', '4', '3', '1', '4', '4', '2', '4', '2', '4', '2', '4', '1', '2']


In [ ]:
fin_preds_lesser_1_int = list(map(int, fin_preds_lesser_1))

In [ ]:
print(fin_preds_lesser_1_int)

[4, 0, 4, 2, 2, 0, 4, 4, 2, 1, 2, 4, 0, 4, 2, 2, 2, 4, 4, 4, 2, 2, 4, 1, 2, 4, 2, 1, 4, 4, 2, 4, 4, 2, 2, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, 1, 2, 1, 4, 4, 4, 4, 4, 3, 1, 4, 4, 2, 4, 2, 4, 2, 4, 1, 2]


In [ ]:
inds_lesser_1=[20, 21, 25, 28, 38, 40, 56, 63, 68, 75, 76, 81, 90, 91, 120, 121, 132, 151, 156, 164, 165, 182, 192, 194, 204, 230, 233, 239, 243, 252, 253, 259, 267, 293, 295, 302, 306, 331, 333, 339, 342, 352, 360, 372, 385, 387, 418, 421, 425, 430, 434, 447, 454, 491, 495, 500, 524, 530, 532, 534, 539, 542, 557, 562, 585, 598]

In [ ]:
len(inds_lesser_1)

66

In [ ]:
len(fin_preds_lesser_1_int)

66

In [ ]:
preds_lesser_2="aarray([4]), array([0]), array([4]), array([2]), array([0]), array([4]), array([1]), array([2]), array([1]), array([4]), array([0]), array([4]), array([2]), array([2]), array([4]), array([4]), array([4]), array([2]), array([2]), array([4]), array([1]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([4]), array([2]), array([1]), array([1]), array([4]), array([4]), array([4]), array([4]), array([3]), array([0]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([2])"

In [ ]:
chunks_lesser_2 = preds_lesser_2.split(',')

In [ ]:
print(chunks_lesser_2)

['aarray([4])', ' array([0])', ' array([4])', ' array([2])', ' array([0])', ' array([4])', ' array([1])', ' array([2])', ' array([1])', ' array([4])', ' array([0])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([2])', ' array([1])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([3])', ' array([0])', ' array([4])', ' array([4])', ' array([2])', ' array([4])', ' array([4])', ' array([4])', ' array([2])']


In [ ]:
fin_preds_lesser_2 = [chunk_lesser_2[8] for chunk_lesser_2 in chunks_lesser_2]

In [ ]:
print(fin_preds_lesser_2)

['4', '0', '4', '2', '0', '4', '1', '2', '1', '4', '0', '4', '2', '2', '4', '4', '4', '2', '2', '4', '1', '4', '4', '4', '4', '4', '4', '2', '4', '4', '4', '4', '4', '2', '4', '2', '1', '1', '4', '4', '4', '4', '3', '0', '4', '4', '2', '4', '4', '4', '2']


In [ ]:
fin_preds_lesser_2_int = list(map(int, fin_preds_lesser_2))

In [ ]:
print(fin_preds_lesser_2_int)

[4, 0, 4, 2, 0, 4, 1, 2, 1, 4, 0, 4, 2, 2, 4, 4, 4, 2, 2, 4, 1, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4, 2, 1, 1, 4, 4, 4, 4, 3, 0, 4, 4, 2, 4, 4, 4, 2]


In [ ]:
inds_lesser_2=[20, 21, 25, 28, 40, 56, 63, 68, 75, 81, 90, 91, 121, 132, 151, 156, 164, 165, 182, 192, 194, 230, 239, 243, 252, 259, 267, 295, 302, 306, 331, 333, 339, 372, 385, 387, 418, 425, 434, 447, 454, 491, 495, 500, 524, 530, 534, 542, 562, 585, 598]

In [ ]:
len(inds_lesser_2)

51

In [ ]:
len(fin_preds_lesser_2_int)

51

In [ ]:
preds_lesser_3="aarray([2]), array([1]), array([1]), array([2]), array([2]), array([4]), array([2]), array([2]), array([2]), array([1]), array([1]), array([2]), array([4]), array([4]), array([2]), array([1]), array([0]), array([2]), array([2])"

In [ ]:
chunks_lesser_3 = preds_lesser_3.split(',')

In [ ]:
print(chunks_lesser_3)

['aarray([2])', ' array([1])', ' array([1])', ' array([2])', ' array([2])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([1])', ' array([1])', ' array([2])', ' array([4])', ' array([4])', ' array([2])', ' array([1])', ' array([0])', ' array([2])', ' array([2])']


In [ ]:
fin_preds_lesser_3 = [chunk_lesser_3[8] for chunk_lesser_3 in chunks_lesser_3]

In [ ]:
print(fin_preds_lesser_3)

['2', '1', '1', '2', '2', '4', '2', '2', '2', '1', '1', '2', '4', '4', '2', '1', '0', '2', '2']


In [ ]:
fin_preds_lesser_3_int = list(map(int, fin_preds_lesser_3))

In [ ]:
print(fin_preds_lesser_3_int)

[2, 1, 1, 2, 2, 4, 2, 2, 2, 1, 1, 2, 4, 4, 2, 1, 0, 2, 2]


In [ ]:
inds_lesser_3=[28, 63, 75, 121, 132, 164, 165, 182, 192, 194, 239, 295, 302, 331, 387, 418, 425, 454, 598]

In [ ]:
len(inds_lesser_3)

19

In [ ]:
len(fin_preds_lesser_3_int)

19

In [ ]:
preds_lesser_4="aarray([2]), array([1]), array([1]), array([4]), array([2]), array([2]), array([2]), array([1]), array([1]), array([4]), array([4]), array([1]), array([0])"

In [ ]:
chunks_lesser_4 = preds_lesser_4.split(',')

In [ ]:
print(chunks_lesser_4)

['aarray([2])', ' array([1])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([1])', ' array([1])', ' array([4])', ' array([4])', ' array([1])', ' array([0])']


In [ ]:
fin_preds_lesser_4 = [chunk_lesser_4[8] for chunk_lesser_4 in chunks_lesser_4]

In [ ]:
print(fin_preds_lesser_4)

['2', '1', '1', '4', '2', '2', '2', '1', '1', '4', '4', '1', '0']


In [ ]:
fin_preds_lesser_4_int = list(map(int, fin_preds_lesser_4))

In [ ]:
print(fin_preds_lesser_4_int)

[2, 1, 1, 4, 2, 2, 2, 1, 1, 4, 4, 1, 0]


In [ ]:
inds_lesser_4=[28, 63, 75, 164, 165, 182, 192, 194, 239, 302, 331, 418, 425]

In [ ]:
len(inds_lesser_4)

13

In [ ]:
len(fin_preds_lesser_4_int)

13

In [ ]:
preds_lesser_5="aarray([2]), array([1]), array([1]), array([4]), array([2]), array([2]), array([2]), array([1]), array([4]), array([4]), array([1]), array([4]), array([0])"

In [ ]:
chunks_lesser_5 = preds_lesser_5.split(',')

In [ ]:
print(chunks_lesser_5)

['aarray([2])', ' array([1])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([2])', ' array([1])', ' array([4])', ' array([4])', ' array([1])', ' array([4])', ' array([0])']


In [ ]:
fin_preds_lesser_5 = [chunk_lesser_5[8] for chunk_lesser_5 in chunks_lesser_5]

In [ ]:
print(fin_preds_lesser_5)

['2', '1', '1', '4', '2', '2', '2', '1', '4', '4', '1', '4', '0']


In [ ]:
fin_preds_lesser_5_int = list(map(int, fin_preds_lesser_5))

In [ ]:
print(fin_preds_lesser_5_int)

[2, 1, 1, 4, 2, 2, 2, 1, 4, 4, 1, 4, 0]


In [ ]:
inds_lesser_5=[28, 63, 75, 164, 165, 182, 192, 194, 239, 302, 331, 418, 425]

In [ ]:
len(inds_lesser_5)

13

In [ ]:
len(fin_preds_lesser_5_int)

13

In [ ]:
preds_lesser_6="aarray([2]), array([1]), array([1]), array([4]), array([2]), array([2]), array([4]), array([1]), array([1]), array([4]), array([4]), array([4]), array([1])"

In [ ]:
chunks_lesser_6 = preds_lesser_6.split(',')

In [ ]:
print(chunks_lesser_6)

['aarray([2])', ' array([1])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([1])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([1])']


In [ ]:
fin_preds_lesser_6 = [chunk_lesser_6[8] for chunk_lesser_6 in chunks_lesser_6]

In [ ]:
print(fin_preds_lesser_6)

['2', '1', '1', '4', '2', '2', '4', '1', '1', '4', '4', '4', '1']


In [ ]:
fin_preds_lesser_6_int = list(map(int, fin_preds_lesser_6))

In [ ]:
print(fin_preds_lesser_6_int)

[2, 1, 1, 4, 2, 2, 4, 1, 1, 4, 4, 4, 1]


In [ ]:
inds_lesser_6=[28, 63, 75, 164, 165, 182, 192, 194, 239, 302, 331, 418, 425]

In [ ]:
len(inds_lesser_6)

13

In [ ]:
len(fin_preds_lesser_6_int)

13

In [ ]:
preds_lesser_7="aarray([2]), array([1]), array([4]), array([4]), array([2]), array([2]), array([4]), array([1]), array([1]), array([4]), array([1]), array([1]), array([1])"

In [ ]:
chunks_lesser_7 = preds_lesser_7.split(',')

In [ ]:
print(chunks_lesser_7)

['aarray([2])', ' array([1])', ' array([4])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([1])', ' array([1])', ' array([4])', ' array([1])', ' array([1])', ' array([1])']


In [ ]:
fin_preds_lesser_7 = [chunk_lesser_7[8] for chunk_lesser_7 in chunks_lesser_7]

In [ ]:
print(fin_preds_lesser_7)

['2', '1', '4', '4', '2', '2', '4', '1', '1', '4', '1', '1', '1']


In [ ]:
fin_preds_lesser_7_int = list(map(int, fin_preds_lesser_7))

In [ ]:
print(fin_preds_lesser_7_int)

[2, 1, 4, 4, 2, 2, 4, 1, 1, 4, 1, 1, 1]


In [ ]:
inds_lesser_7=[28, 63, 75, 164, 165, 182, 192, 194, 239, 302, 331, 418, 425]

In [ ]:
len(inds_lesser_7)

13

In [ ]:
len(fin_preds_lesser_7_int)

13

In [ ]:
preds_lesser_8="aarray([2]), array([1]), array([1]), array([4]), array([2]), array([2]), array([4]), array([1]), array([4]), array([4]), array([4]), array([4]), array([1])"

In [ ]:
chunks_lesser_8 = preds_lesser_8.split(',')

In [ ]:
print(chunks_lesser_8)

['aarray([2])', ' array([1])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([4])', ' array([1])', ' array([4])', ' array([4])', ' array([4])', ' array([4])', ' array([1])']


In [ ]:
fin_preds_lesser_8 = [chunk_lesser_8[8] for chunk_lesser_8 in chunks_lesser_8]

In [ ]:
print(fin_preds_lesser_8)

['2', '1', '1', '4', '2', '2', '4', '1', '4', '4', '4', '4', '1']


In [ ]:
fin_preds_lesser_8_int = list(map(int, fin_preds_lesser_8))

In [ ]:
print(fin_preds_lesser_8_int)

[2, 1, 1, 4, 2, 2, 4, 1, 4, 4, 4, 4, 1]


In [ ]:
inds_lesser_8=[28, 63, 75, 164, 165, 182, 192, 194, 239, 302, 331, 418, 425]

In [ ]:
len(inds_lesser_8)

13

In [ ]:
len(fin_preds_lesser_8_int)

13

In [ ]:
preds_lesser_9="aarray([2]), array([1]), array([4]), array([2]), array([2]), array([1]), array([4]), array([1]), array([1])"

In [ ]:
chunks_lesser_9 = preds_lesser_9.split(',')

In [ ]:
print(chunks_lesser_9)

['aarray([2])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([1])', ' array([4])', ' array([1])', ' array([1])']


In [ ]:
fin_preds_lesser_9 = [chunk_lesser_9[8] for chunk_lesser_9 in chunks_lesser_9]

In [ ]:
print(fin_preds_lesser_9)

['2', '1', '4', '2', '2', '1', '4', '1', '1']


In [ ]:
fin_preds_lesser_9_int = list(map(int, fin_preds_lesser_9))

In [ ]:
print(fin_preds_lesser_9_int)

[2, 1, 4, 2, 2, 1, 4, 1, 1]


In [ ]:
inds_lesser_9=[28, 63, 164, 165, 182, 194, 239, 418, 425]

In [ ]:
len(inds_lesser_9)

9

In [ ]:
len(fin_preds_lesser_9)

9

In [ ]:
preds_lesser_10="aarray([2]), array([1]), array([4]), array([2]), array([2]), array([1]), array([1]), array([1]), array([1])"

In [ ]:
chunks_lesser_10 = preds_lesser_10.split(',')

In [ ]:
print(chunks_lesser_10)

['aarray([2])', ' array([1])', ' array([4])', ' array([2])', ' array([2])', ' array([1])', ' array([1])', ' array([1])', ' array([1])']


In [ ]:
fin_preds_lesser_10 = [chunk_lesser_10[8] for chunk_lesser_10 in chunks_lesser_10]

In [ ]:
print(fin_preds_lesser_10)

['2', '1', '4', '2', '2', '1', '1', '1', '1']


In [ ]:
fin_preds_lesser_10_int = list(map(int, fin_preds_lesser_10))

In [ ]:
print(fin_preds_lesser_10_int)

[2, 1, 4, 2, 2, 1, 1, 1, 1]


In [ ]:
inds_lesser_10=[28, 63, 164, 165, 182, 194, 239, 418, 425]

In [ ]:
len(inds_lesser_10)

9

In [ ]:
len(fin_preds_lesser_10_int)

9

In [ ]:
preds_lesser_11="aarray([2]), array([4]), array([2]), array([2]), array([1]), array([1])"

In [ ]:
chunks_lesser_11 = preds_lesser_11.split(',')

In [ ]:
print(chunks_lesser_11)

['aarray([2])', ' array([4])', ' array([2])', ' array([2])', ' array([1])', ' array([1])']


In [ ]:
fin_preds_lesser_11 = [chunk_lesser_11[8] for chunk_lesser_11 in chunks_lesser_11]

In [ ]:
print(fin_preds_lesser_11)

['2', '4', '2', '2', '1', '1']


In [ ]:
fin_preds_lesser_11_int = list(map(int, fin_preds_lesser_11))

In [ ]:
print(fin_preds_lesser_11_int)

[2, 4, 2, 2, 1, 1]


In [ ]:
inds_lesser_11=[28, 164, 165, 182, 194, 239]

In [ ]:
len(inds_lesser_11)

6

In [ ]:
len(fin_preds_lesser_11_int)

6

In [ ]:
preds_lesser_12="aarray([4]), array([1]), array([1])"

In [ ]:
chunks_lesser_12 = preds_lesser_12.split(',')

In [ ]:
print(chunks_lesser_12)

['aarray([4])', ' array([1])', ' array([1])']


In [ ]:
fin_preds_lesser_12 = [chunk_lesser_12[8] for chunk_lesser_12 in chunks_lesser_12]

In [ ]:
print(fin_preds_lesser_12)

['4', '1', '1']


In [ ]:
fin_preds_lesser_12_int = list(map(int, fin_preds_lesser_12))

In [ ]:
print(fin_preds_lesser_12_int)

[4, 1, 1]


In [ ]:
inds_lesser_12=[164, 194, 239]

In [ ]:
len(inds_lesser_12)

3

In [ ]:
len(fin_preds_lesser_12_int)

3

In [ ]:
#Final predictions for rows < 0.7
fin_inds=[20, 
21, 
25, 
28, 
38, 
40, 
56, 
63, 
68, 
75, 
76, 
81, 
90, 
91, 120, 121, 132, 151, 156, 164, 165, 182, 192, 194, 204, 230, 233, 239, 243, 252, 253, 259, 267, 293, 295, 302, 306, 331, 333, 339, 342, 352, 360, 372, 385, 387, 418, 421, 425, 430, 434, 447, 454, 491, 495, 500, 524, 530, 532, 534, 539, 542, 557, 562, 585, 598
]

In [ ]:
fin_preds=[4, 
0, 
4,
2,
2,
0, 
4,
1,
2,
1,
2,
4, 
0, 
4,
2,
2,
2,
2,
4, 
4, 
2,
2,
4,
1,
2,
4,
2,
1,
4,
4, 
4, 
4, 
2,
2,
2,
1,
4,
1,
4, 
4,
2, 
2, 
2,
2,
4,
2,
1,
2,
1,
4,
4,
4,
2,
4, 
3, 
0, 
4, 
4,
2,
2,
2,
4,
2,
4,
4,
2,
]


In [ ]:
len(fin_inds)

66

In [ ]:
len(fin_preds)

66

In [ ]:
all_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 234, 235, 236, 237, 238, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 251, 254, 255, 256, 257, 258, 260, 261, 262, 263, 264, 265, 266, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294, 296, 297, 298, 299, 300, 301, 303, 304, 305, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 332, 334, 335, 336, 337, 338, 340, 341, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 356, 357, 358, 359, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 386, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 419, 420, 422, 423, 424, 426, 427, 428, 429, 431, 432, 433, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 448, 449, 450, 451, 452, 453, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 492, 493, 494, 496, 497, 498, 499, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 525, 526, 527, 528, 529, 531, 533, 535, 536, 537, 538, 540, 541, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 558, 559, 560, 561, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613,20, 
21, 
25, 
28, 
38, 
40, 
56, 
63, 
68, 
75, 
76, 
81, 
90, 91, 120, 121, 132, 151, 156, 164, 165, 182, 192, 194, 204, 230, 233, 239, 243, 252, 253, 259, 267, 293, 295, 302, 306, 331, 333, 339, 342, 352, 360, 372, 385, 387, 418, 421, 425, 430, 434, 447, 454, 491, 495, 500, 524, 530, 532, 534, 539, 542, 557, 562, 585, 598]

In [ ]:
all_preds=[2, 2, 0, 0, 2, 0, 0, 2, 4, 0, 0, 1, 3, 0, 0, 2, 2, 2, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 4, 2, 0, 0, 2, 3, 0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 3, 0, 0, 0, 0, 2, 2, 0, 3, 2, 0, 3, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 3, 2, 3, 2, 2, 3, 2, 4, 2, 3, 0, 3, 3, 0, 0, 3, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 2, 2, 0, 2, 0, 4, 0, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, 2, 2, 0, 3, 2, 0, 4, 0, 0, 2, 3, 2, 2, 0, 3, 0, 2, 0, 0, 3, 0, 0, 2, 0, 0, 2, 2, 0, 0, 4, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0, 0, 3, 0, 4, 0, 3, 4, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 4, 2, 2, 2, 0, 0, 0, 0, 0, 2, 4, 0, 3, 0, 0, 0, 0, 2, 4, 2, 4, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 4, 3, 2, 4, 3, 2, 0, 0, 3, 0, 4, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 3, 0, 0, 0, 4, 4, 2, 4, 2, 2, 0, 2, 2, 0, 4, 2, 2, 3, 0, 3, 0, 2, 2, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 2, 2, 3, 0, 2, 0, 2, 0, 2, 2, 4, 2, 0, 0, 2, 0, 4, 0, 3, 2, 0, 2, 3, 4, 2, 0, 0, 2, 2, 3, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 4, 0, 0, 0, 2, 2, 0, 2, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 4, 2, 0, 2, 2, 0, 4, 4, 0, 0, 2, 3, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 4, 2, 3, 0, 0, 0, 0, 3, 0, 2, 2, 3, 2, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 3, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 4, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 3, 3, 2, 2, 3, 2, 3, 0, 0, 0, 2, 0, 4, 2, 3, 3, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 4, 4, 0, 2, 2, 3, 2, 3, 0, 2, 4, 2, 2, 2, 0, 4, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 4, 2, 0, 0, 2, 2, 2, 2, 2, 3, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0,4, 
0, 
4,
2,
2,
0, 
4,
1,
2,
1,
2,
4, 
0, 
4,
2,
2,
2,
2,
4, 
4, 
2,
2,
4,
1,
2,
4,
2,
1,
4,
4, 
4, 
4, 
2,
2,
2,
1,
4,
1,
4, 
4,
2, 
2, 
2,
2,
4,
2,
1,
2,
1,
4,
4,
4,
2,
4, 
3, 
0, 
4, 
4,
2,
2,
2,
4,
2,
4,
4,
2,
]

In [ ]:
len(all_indices)

614

In [ ]:
len(all_preds)

614

In [ ]:
y_test_dt

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
type(y_test_dt)

pandas.core.series.Series

In [ ]:
all_preds_ser=pd.Series(all_preds)

In [ ]:
all_preds_ser

0      2
1      2
2      0
3      0
4      2
      ..
609    4
610    2
611    4
612    4
613    2
Length: 614, dtype: int64

In [ ]:
all_preds_ser.reindex(all_indices) #matching the predictions and the indices (matching all_preds_ser and all_indices)

0      2
1      2
2      0
3      0
4      2
      ..
542    2
557    1
562    2
585    1
598    4
Length: 614, dtype: int64

In [ ]:
y_test_dt #original y_test

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
y_test_cp=y_test_dt.copy()

In [ ]:
new_y_test=y_test_cp.reindex(all_indices) #reordering y_test_dt according to all_preds_ser 's order

In [ ]:
new_y_test

0      2
1      2
2      0
3      0
4      2
      ..
542    1
557    2
562    1
585    4
598    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(new_y_test, all_preds_ser)

0.8550488599348535

In [ ]:
#xxxxxxxxx done xxxxxxxx

In [ ]:
n=np.array([array([1]), array([4])], dtype=list)
#np.array([(str("u"), [1,2,3])], dtype=..)

In [ ]:
from array import array

In [ ]:
a="array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([1]), array([3]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([3]), array([0]), array([2]), array([0]), array([0]), array([3]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([3]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([3]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([3]), array([2]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([3]), array([2]), array([3]), array([2]), array([2]), array([3]), array([2]), array([2]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([2]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([2]), array([4]), array([2]), array([2]), array([0]), array([3]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([3]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([4]), array([0]), array([2]), array([0]), array([3]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([2]), array([0]), array([3]), array([0]), array([0]), array([3]), array([0]), array([4]), array([0]), array([3]), array([4]), array([0]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([2]), array([2]), array([4]), array([3]), array([2]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([3]), array([4]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([4]), array([2]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([2]), array([3]), array([0]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([4]), array([2]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([3]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([2]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([0]), array([2]), array([4]), array([0]), array([0]), array([2]), array([0]), array([4]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([4]), array([0]), array([2]), array([3]), array([0]), array([3]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([4]), array([2]), array([3]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([2]), array([2]), array([3]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([4]), array([0]), array([3]), array([0]), array([0]), array([0]), array([3]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([3]), array([3]), array([2]), array([2]), array([3]), array([2]), array([4]), array([3]), array([0]), array([0]), array([0]), array([2]), array([0]), array([2]), array([3]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([2]), array([3]), array([2]), array([3]), array([0]), array([2]), array([4]), array([2]), array([2]), array([2]), array([0]), array([4]), array([3]), array([0]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([2]), array([3]), array([2]), array([0]), array([2]), array([2]), array([0]), array([0]), array([2]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([4]), array([4]), array([0]), array([4]), array([4]), array([4]), array([3]), array([2]), array([4]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([2]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([2]), array([3]), array([2]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([4]), array([4]), array([0]), array([4]), array([4]), array([4]), array([3]), array([1]), array([4]), array([4]), array([2]), array([4]), array([4]), array([4]), array([4]), array([1]), array([2]), array([4]), array([2]), array([2]), array([2]), array([1]), array([2]), array([1]), array([4])"

In [ ]:
print(a)

array([2]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([0]), array([1]), array([3]), array([0]), array([0]), array([2]), array([2]), array([2]), array([0]), array([0]), array([0]), array([0]), array([0]), array([3]), array([0]), array([4]), array([3]), array([0]), array([2]), array([0]), array([0]), array([3]), array([0]), array([2]), array([0]), array([0]), array([2]), array([0]), array([3]), array([2]), array([0]), array([0]), array([0]), array([2]), array([0]), array([4]), array([2]), array([0]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([0]), array([4]), array([0]), array([0]), array([0]), array([2]), array([0]), array([0]), array([2]), array([3]), array([0]), array([2]), array([0]), array([0]), array([0]), array([2]), array([2]), array([0]), array([4]), array([2]), array([3]), array([2]), array([0]), array([3]), array([0]), array([2]), array([2]), array([0]), array([0]), array([0]), arra

In [ ]:
chunks = a.split(',')

In [ ]:
chunks

['array([2])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([1])',
 ' array([3])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([2])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([3])',
 ' array([0])',
 ' array([4])',
 ' array([3])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([3])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([3])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([4])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([4])',
 ' array([0])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array([0])',
 ' array([0])',
 ' array([2])',
 ' array(

In [ ]:
letters = [chunk[8] for chunk in chunks]

In [ ]:
final_letters=letters[1:]

In [ ]:
letters

[']',
 '2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '1',
 '3',
 '0',
 '0',
 '2',
 '2',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '3',
 '0',
 '4',
 '3',
 '0',
 '2',
 '0',
 '0',
 '3',
 '0',
 '2',
 '0',
 '0',
 '2',
 '0',
 '3',
 '2',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '2',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '4',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '2',
 '3',
 '0',
 '2',
 '0',
 '0',
 '0',
 '2',
 '2',
 '0',
 '4',
 '2',
 '3',
 '2',
 '0',
 '3',
 '0',
 '2',
 '2',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '0',
 '0',
 '0',
 '2',
 '0',
 '2',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '3',
 '2',
 '3',
 '2',
 '2',
 '3',
 '2',
 '2',
 '3',
 '0',
 '3',
 '0',
 '0',
 '0',
 '2',
 '2',
 '3',
 '0',
 '0',
 '0',
 '0',
 '3',
 '0',
 '0',
 '0',
 '3',
 '0',
 '0',
 '2',
 '2',
 '0',
 '2',
 '0',
 '0',
 '2',
 '2',
 '0',
 '2',
 '2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '4',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '2',
 '2',
 '4',
 '2',
 '2',
 '0',
 '3',
 '2',
 '0',
 '2',
 '0',
 '0',
 '2',
 '3'

In [ ]:
final_letters

['2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '1',
 '3',
 '0',
 '0',
 '2',
 '2',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '3',
 '0',
 '4',
 '3',
 '0',
 '2',
 '0',
 '0',
 '3',
 '0',
 '2',
 '0',
 '0',
 '2',
 '0',
 '3',
 '2',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '2',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '4',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '2',
 '3',
 '0',
 '2',
 '0',
 '0',
 '0',
 '2',
 '2',
 '0',
 '4',
 '2',
 '3',
 '2',
 '0',
 '3',
 '0',
 '2',
 '2',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '0',
 '0',
 '0',
 '2',
 '0',
 '2',
 '0',
 '2',
 '0',
 '0',
 '0',
 '0',
 '0',
 '2',
 '3',
 '2',
 '3',
 '2',
 '2',
 '3',
 '2',
 '2',
 '3',
 '0',
 '3',
 '0',
 '0',
 '0',
 '2',
 '2',
 '3',
 '0',
 '0',
 '0',
 '0',
 '3',
 '0',
 '0',
 '0',
 '3',
 '0',
 '0',
 '2',
 '2',
 '0',
 '2',
 '0',
 '0',
 '2',
 '2',
 '0',
 '2',
 '2',
 '0',
 '0',
 '2',
 '0',
 '0',
 '4',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '4',
 '2',
 '2',
 '4',
 '2',
 '2',
 '0',
 '3',
 '2',
 '0',
 '2',
 '0',
 '0',
 '2',
 '3',
 '2'

In [ ]:
final_letters = list(map(int, final_letters))

In [ ]:
final_letters

[2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 3,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 4,
 3,
 0,
 2,
 0,
 0,
 3,
 0,
 2,
 0,
 0,
 2,
 0,
 3,
 2,
 0,
 0,
 0,
 2,
 0,
 4,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 3,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 4,
 2,
 3,
 2,
 0,
 3,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 4,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 3,
 0,
 3,
 0,
 0,
 0,
 2,
 2,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 2,
 0,
 4,
 2,
 2,
 4,
 2,
 2,
 0,
 3,
 2,
 0,
 2,
 0,
 0,
 2,
 3,
 2,
 2,
 0,
 3,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 4,
 0,
 2,
 0,
 3,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 3,
 0,
 2,
 2,
 2,
 0,
 3,
 0,
 0,
 3,
 0,
 4,
 0,
 3,
 4,
 0,
 0,
 0,
 2,
 2,
 3,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 3,
 0,
 0,
 0,
 2,
 0,


In [ ]:
len(final_letters)

613

In [ ]:
y_test_dt

0      2
1      2
2      0
3      0
4      2
      ..
609    2
610    0
611    0
612    0
613    0
Name: 3000, Length: 614, dtype: int64

In [ ]:
import pandas as pd
df=pd.DataFrame()
df[0]=[100,200,300]
df[1]=[400,500,600]

In [ ]:
print(df)

     0    1
0  100  400
1  200  500
2  300  600


In [ ]:
df[1:3]

,0,1
1,200,500
2,300,600


In [ ]:
new_df=pd.DataFrame()

In [ ]:
for row in df[1:3].itertuples(index=True):
      new_df=new_df.append(pd.DataFrame(row))

In [ ]:
new_df

,0
0,1
1,200
2,500
0,2
1,300
2,600


In [ ]:
for index, row in df[1:3].iterrows():
      new_df=new_df.append(row,ignore_index=False)

In [ ]:
new_df

,0,1
1,200.0,500.0
2,300.0,600.0


In [ ]:
#"/content/gdrive/My Drive/physionet/ensemble/tri_training/50_epochs/model_i/teacher_i_wts.ckpt"
with open("/content/gdrive/My Drive/physionet/ensemble/batching_wrk/outputs.txt", "a") as out:

  out.write(batching(x_test_dt[0:615],student_model) +'\n')
  out.close()

round  1
[11, 23, 38, 40, 56, 60, 63, 68, 75, 76, 95, 105, 113, 117, 123, 132, 135, 143, 152, 156, 157, 182, 188, 194, 230, 233, 235, 239, 252, 267, 295, 306, 331, 332, 334, 336, 339, 342, 360, 372, 377, 385, 387, 391, 392, 407, 434, 439, 447, 481, 495, 500, 513, 514, 530, 534, 542, 546, 563, 577, 585, 598, 599]
length of batch df is  63
Model: "model_1847"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)

OpError: ignored

In [ ]:
import sys

print('This message will be displayed on the screen.')

original_stdout = sys.stdout # Save a reference to the original standard output

with open('/content/gdrive/My Drive/physionet/ensemble/batching_wrk/outputs.txt', 'a') as f: #'w' model will rewrite data to the file. 'a' will append data to a file, if it is the first time writing data to the file, then it will create a new file and then add/append data to it
    sys.stdout = f # Change the standard output to the file we created.
    print('This message will be written to a file.')
    sys.stdout = original_stdout # Reset the standard output to its original value

This message will be displayed on the screen.


In [ ]:
original_stdout = sys.stdout 
for i in range(5):
  
  

  with open('/content/gdrive/My Drive/physionet/ensemble/batching_wrk/outputs.txt', 'a') as f: 
    sys.stdout = f # Change the standard output to the file we created.
    print(i)
    sys.stdout = original_stdout # Reset the standard output to its original value

In [ ]:
import sys

In [ ]:
a,b =batching(x_test_dt[0:615],student_model)

Streaming output truncated to the last 5000 lines.
 cConv2 (Conv1D)                (None, 15, 128)      24704       ['cDrop1[0][0]']                 
                                                                                                  
 fConv3 (Conv1D)                (None, 62, 128)      131200      ['fConv2[0][0]']                 
                                                                                                  
 cConv3 (Conv1D)                (None, 15, 128)      98432       ['cConv2[0][0]']                 
                                                                                                  
 fConv4 (Conv1D)                (None, 62, 128)      131200      ['fConv3[0][0]']                 
                                                                                                  
 cConv4 (Conv1D)                (None, 15, 128)      98432       ['cConv3[0][0]']                 
                                                          

KeyboardInterrupt: ignored

In [ ]:
student_preds

[]

In [ ]:
#round_cnt=0
return_counter=0
student_preds=[]


def batching(x_test,student_mod):

  batch_df=pd.DataFrame()
  indices=[]
  
  global return_counter, student_preds
  
  

  for sple in np.array(x_test):
    
    sple_rshp=sple.reshape(1,3000,1)
    sple_df_rshp=sple.reshape(1,3000)
    sple_df_rshp_df=pd.DataFrame(sple_df_rshp)

    if return_counter < 1:
      
      student_i_probs=student_model.predict(sple_rshp)
      student_i_max_prob=student_i_probs.max()
      return_counter=return_counter+1

      if student_i_max_prob < 0.7:
        
        batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))

        #indices=indices.append(sple_df_rshp_df.index)
        
      else:
        
        student_i_fin=np.argmax(student_i_probs,axis=1)

        student_preds.append(student_i_fin)

      #if not batch_df.empty:
        
        #round_cnt=round_cnt+1

      #print("the round number is",round_cnt)


    else:

      student_i_probs=student_mod.predict(sple_rshp)
      student_i_max_prob=student_i_probs.max()

      if student_i_max_prob < 0.7:
        
        batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))

        #indices=indices.append(sple_df_rshp.index)
        
      else:
        
        student_i_fin=np.argmax(student_i_probs,axis=1)

        student_preds.append(student_i_fin)

      #if not batch_df.empty:
        
        #round_cnt=round_cnt+1
        #print("the round number is",round_cnt)

      
    
  #while batch_df.empty == False:

  if not batch_df.empty:
    
    len_df=len(batch_df)

    preds_shuf1=[]
    preds_inc_shuf1=[]

    preds_shuf2=[]
    preds_inc_shuf2=[]

    preds_shuf3=[]
    preds_inc_shuf3=[]

    m_vote_inc=[]
    m_vote=[]

    #global preds_shuf1, preds_inc_shuf1, preds_shuf2, preds_inc_shuf2, preds_shuf3, preds_inc_shuf3, m_vote_inc, m_vote




    for sple in np.array(batch_df[0:len_df]):

      

      
      
      sple_rshp=sple.reshape(1,3000,1)

      p1_i=loaded_model_shuf1.predict(sple_rshp)
    
      p1_i_fin=np.argmax(p1_i,axis=1)
    
      preds_shuf1.append(p1_i_fin)


      latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)
    
      loaded_model_shuf1_inc = model_b(verbose=VBS)

      loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

      loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p1_i_inc=loaded_model_shuf1_inc.predict(sple_rshp)

      p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)


      p2_i=loaded_model_shuf2.predict(sple_rshp)

      p2_i_fin=np.argmax(p2_i,axis=1)

      preds_shuf2.append(p2_i_fin)


      latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

      loaded_model_shuf2_inc = model_b(verbose=VBS)

      loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

      loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p2_i_inc=loaded_model_shuf2_inc.predict(sple_rshp)


      p3_i=loaded_model_shuf3.predict(sple_rshp)

      p3_i_fin=np.argmax(p3_i,axis=1)

      preds_shuf3.append(p3_i_fin)


      latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

      loaded_model_shuf3_inc = model_b(verbose=VBS)

      loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

      loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p3_i_inc=loaded_model_shuf3_inc.predict(sple_rshp)


      if (p1_i_inc != p1_i).all():
        
        p1_i_inc=p1_i

      if (p2_i_inc != p2_i).all():
        
        p2_i_inc=p2_i

      if (p3_i_inc != p3_i).all():
         
        p3_i_inc=p3_i

      if (p2_i_inc==p3_i_inc).all():
          
        p1_i_inc=p3_i_inc
        
        p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1) 

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])


        preds_inc_shuf2.append(p2_i_inc_fin)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)

      elif (p3_i_inc==p1_i_inc).all():
        
        p2_i_inc=p1_i_inc

        p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

        preds_inc_shuf2.append(p2_i_inc_fin)


        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])


        preds_inc_shuf1.append(p1_i_inc_fin)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)

      elif (p2_i_inc==p1_i_inc).all(): 
        
        p3_i_inc=p1_i_inc

        p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)


        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

    
        preds_inc_shuf2.append(p2_i_inc_fin)

      else:
        
        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

        preds_inc_shuf2.append(p2_i_inc_fin)


        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])


        preds_inc_shuf3.append(p3_i_inc_fin)


      mode_i_inc=mode([p1_i_inc_fin,p2_i_inc_fin,p3_i_inc_fin])
      m_vote_i_inc=mode_i_inc[0][0]
      m_vote_inc.append(m_vote_i_inc)

      mode_i=mode([p1_i_fin,p2_i_fin,p3_i_fin])
      m_vote_i=mode_i[0][0]
      m_vote.append(m_vote_i)

    batch_df_cp=batch_df.copy()
    batch_df_cp['sleep_stage_voted']=m_vote_inc
    batch_df_x=batch_df.copy()

    batch_df_x_rshp=np.array(batch_df_x).reshape(len(batch_df_x),3000,1)

    batch_df_y_=np_utils.to_categorical(m_vote_inc)

    teacher_model_i=model_b(verbose=VBS) #50 epochs teacher model

    teacher_model_i.load_weights(teacher_checkpt_path)
    teacher_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    teacher_model_i.fit(batch_df_x_rshp, batch_df_y_, epochs=50, callbacks=[cp_callback_teacher_model_i]) #write code in the previous cell to checkpoint every teacher model that is trained to every chunk of the batch_df data frame
    #fit the teacher model according to the batch df chunk and check point it

    student_model_i=get_model_1d_cnn()
    student_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



    # Initialize and compile distiller
    distiller_i = Distiller(student=student_model_i, teacher=teacher_model_i)
    distiller_i.compile(optimizer=keras.optimizers.Adam(),
                        
                        metrics=[keras.metrics.CategoricalAccuracy()],
                        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True), #perform kd on the student model
                        distillation_loss_fn=keras.losses.KLDivergence(),
                        alpha=0.9,temperature=5)

    # Distill teacher to student
    #distiller.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])
    distiller_i.fit(batch_df_x_rshp, batch_df_y_, epochs=60)

    return batching(batch_df_x,student_model_i) 



  #return batching(batching_df_x,student_model_i)  #call the batching() by passing batch_df_x




       

       
       #use the student model to perform preds on the batch df one at a time(go batck to the first line of the function)
       #check confidence, generate batch df....and continue until batch df is empty





















In [ ]:
batching(x_test_dt[0:615],student_model)

Model: "model_33"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                           

UnboundLocalError: ignored

In [ ]:
round_cnt=0
def batching(x_test):

  batch_df=pd.DataFrame()
  indices=[]
  

  for sple in np.array(x_test):
    
    sple_rshp=sple.reshape(1,3000,1)
    sple_df_rshp=sple.reshape(1,3000)
    student_i_probs=student_model.predict(sple_rshp)
    student_i_max_prob=student_i_probs.max()

    if student_i_max_prob < 0.7:
      
      batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))

      indices=indices.append(sple_df_rshp.index)

    else:
      
      student_i_fin=np.argmax(student_i_probs,axis=1)

      student_preds.append(student_i_fin)

    if not batch_df.empty:

      round_cnt=round_cnt+1
      print("the round number is",round_cnt)

      
    
  while batch_df.empty == False:

    if not batch_df.empty:
      
      len_df=len(batch_df)+1

      preds_shuf1=[]
      preds_inc_shuf1=[]

      preds_shuf2=[]
      preds_inc_shuf2=[]

      preds_shuf3=[]
      preds_inc_shuf3=[]

      m_vote_inc=[]
      m_vote=[]




      for sple in np.array(batch_df[0:len_df]):
        
        sple_rshp=sple.reshape(1,3000,1)

        p1_i=loaded_model_shuf1.predict(sple_rshp)
    
        p1_i_fin=np.argmax(p1_i,axis=1)
    
        preds_shuf1.append(p1_i_fin)


        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)
    
        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        p1_i_inc=loaded_model_shuf1_inc.predict(sple_rshp)

        p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)


        p2_i=loaded_model_shuf2.predict(sple_rshp)

        p2_i_fin=np.argmax(p2_i,axis=1)

        preds_shuf2.append(p2_i_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        p2_i_inc=loaded_model_shuf2_inc.predict(sple_rshp)


        p3_i=loaded_model_shuf3.predict(sple_rshp)

        p3_i_fin=np.argmax(p3_i,axis=1)

        preds_shuf3.append(p3_i_fin)


        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
        p3_i_inc=loaded_model_shuf3_inc.predict(sple_rshp)


        if (p1_i_inc != p1_i).all():
          
          p1_i_inc=p1_i

        if (p2_i_inc != p2_i).all():
        
          p2_i_inc=p2_i

        if (p3_i_inc != p3_i).all():
         
          p3_i_inc=p3_i

        if (p2_i_inc==p3_i_inc).all():
          
          p1_i_inc=p3_i_inc
        
          p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

          latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1) 

          loaded_model_shuf1_inc = model_b(verbose=VBS)

          loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

          loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

          preds_inc_shuf1.append(p1_i_inc_fin)


          latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

          loaded_model_shuf2_inc = model_b(verbose=VBS)

          loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

          loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])


          preds_inc_shuf2.append(p2_i_inc_fin)

          latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

          loaded_model_shuf3_inc = model_b(verbose=VBS)

          loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

          loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

          preds_inc_shuf3.append(p3_i_inc_fin)

        elif (p3_i_inc==p1_i_inc).all():
          
          p2_i_inc=p1_i_inc

          p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

          latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

          loaded_model_shuf2_inc = model_b(verbose=VBS)

          loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

          loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

          preds_inc_shuf2.append(p2_i_inc_fin)


          latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

          loaded_model_shuf1_inc = model_b(verbose=VBS)

          loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

          loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])


          preds_inc_shuf1.append(p1_i_inc_fin)

          latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

          loaded_model_shuf3_inc = model_b(verbose=VBS)

          loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

          loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

          preds_inc_shuf3.append(p3_i_inc_fin)

        elif (p2_i_inc==p1_i_inc).all(): 
          
          p3_i_inc=p1_i_inc

          p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

          latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

          loaded_model_shuf3_inc = model_b(verbose=VBS)

          loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

          loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

          preds_inc_shuf3.append(p3_i_inc_fin)


          latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

          loaded_model_shuf1_inc = model_b(verbose=VBS)

          loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

          loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

          preds_inc_shuf1.append(p1_i_inc_fin)


          latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

          loaded_model_shuf2_inc = model_b(verbose=VBS)

          loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

          loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

    
          preds_inc_shuf2.append(p2_i_inc_fin)

        else:
          
          latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

          loaded_model_shuf1_inc = model_b(verbose=VBS)

          loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

          loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

          preds_inc_shuf1.append(p1_i_inc_fin)


          latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

          loaded_model_shuf2_inc = model_b(verbose=VBS)

          loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

          loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

          preds_inc_shuf2.append(p2_i_inc_fin)


          latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

          loaded_model_shuf3_inc = model_b(verbose=VBS)

          loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

          loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
          loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])


          preds_inc_shuf3.append(p3_i_inc_fin)


        mode_i_inc=mode([p1_i_inc_fin,p2_i_inc_fin,p3_i_inc_fin])
        m_vote_i_inc=mode_i_inc[0][0]
        m_vote_inc.append(m_vote_i_inc)

        mode_i=mode([p1_i_fin,p2_i_fin,p3_i_fin])
        m_vote_i=mode_i[0][0]
        m_vote.append(m_vote_i)

      batch_df_cp=batch_df.copy()
      batch_df_cp['sleep_stage_voted']=m_vote_inc
      batch_df_x=batch_df.copy()

      batch_df_x_rshp=np.array(batch_df_x).reshape(len(batch_df_x),3000,1)

      batch_df_y_=np_utils.to_categorical(m_vote_inc)

      teacher_model_i=model_b(verbose=VBS) #50 epochs teacher model

      teacher_model_i.load_weights(teacher_checkpt_path)
      teacher_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

      teacher_model_i.fit(batch_df_x_rshp, batch_df_y_, epochs=50, callbacks=[cp_callback_teacher_model_i]) #write code in the previous cell to checkpoint every teacher model that is trained to every chunk of the batch_df data frame
      #fit the teacher model according to the batch df chunk and check point it

      student_model_i=get_model_1d_cnn()
      student_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



      # Initialize and compile distiller
      distiller_i = Distiller(student=student_model_i, teacher=teacher_model_i)
      distiller_i.compile(optimizer=keras.optimizers.Adam(),
                          metrics=[keras.metrics.CategoricalAccuracy()],
                         student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True), #perform kd on the student model
                         distillation_loss_fn=keras.losses.KLDivergence(),
                         alpha=0.9,temperature=5)

      # Distill teacher to student
      #distiller.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])
      distiller_i.fit(batch_df_x_rshp, batch_df_y_, epochs=60)

  batching(batching_df_x)  #call the batching() by passing batch_df_x




       

       
       #use the student model to perform preds on the batch df one at a time(go batck to the first line of the function)
       #check confidence, generate batch df....and continue until batch df is empty





















In [ ]:
def batching(x_test):

  for sple in np.array(x_test):
    
    sple_rshp=sple.reshape(1,3000,1)
    sple_df_rshp=sple.reshape(1,3000)
    student_i_probs=student_model.predict(sple_rshp)
    student_i_max_prob=student_i_probs.max()

    if student_i_max_prob < 0.7:
      
      batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))

      indices=indices.append(sple_df_rshp.index)

    else:
      
      student_i_fin=np.argmax(student_i_probs,axis=1)

      student_preds.append(student_i_fin)
    
  while batch_df.empty == False:
    

  if not batch_df.empty:

    len_df=len(batch_df)+1

    for sple in np.array(batch_df[0:len_df]):

      sple_rshp=sple.reshape(1,3000,1)

      p1_i=loaded_model_shuf1.predict(sple_rshp)
    
      p1_i_fin=np.argmax(p1_i,axis=1)
    
      preds_shuf1.append(p1_i_fin)


      latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)
    
      loaded_model_shuf1_inc = model_b(verbose=VBS)

      loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

      loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p1_i_inc=loaded_model_shuf1_inc.predict(sple_rshp)

      p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)


      p2_i=loaded_model_shuf2.predict(sple_rshp)

      p2_i_fin=np.argmax(p2_i,axis=1)

      preds_shuf2.append(p2_i_fin)


      latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

      loaded_model_shuf2_inc = model_b(verbose=VBS)

      loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

      loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p2_i_inc=loaded_model_shuf2_inc.predict(sple_rshp)


      p3_i=loaded_model_shuf3.predict(sple_rshp)

      p3_i_fin=np.argmax(p3_i,axis=1)

      preds_shuf3.append(p3_i_fin)


      latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

      loaded_model_shuf3_inc = model_b(verbose=VBS)

      loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

      loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
      p3_i_inc=loaded_model_shuf3_inc.predict(sple_rshp)


      if (p1_i_inc != p1_i).all():
        
        p1_i_inc=p1_i

      if (p2_i_inc != p2_i).all():
        
        p2_i_inc=p2_i

      if (p3_i_inc != p3_i).all():
         
        p3_i_inc=p3_i

      if (p2_i_inc==p3_i_inc).all():
        
        p1_i_inc=p3_i_inc
        
        p1_i_inc_fin=np.argmax(p1_i_inc,axis=1)

        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])


        preds_inc_shuf2.append(p2_i_inc_fin)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)

      elif (p3_i_inc==p1_i_inc).all():
        
        p2_i_inc=p1_i_inc

        p2_i_inc_fin=np.argmax(p2_i_inc,axis=1)

        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

        preds_inc_shuf2.append(p2_i_inc_fin)


        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])


        preds_inc_shuf1.append(p1_i_inc_fin)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)

      elif (p2_i_inc==p1_i_inc).all(): 
        
        p3_i_inc=p1_i_inc

        p3_i_inc_fin=np.argmax(p3_i_inc,axis=1)

        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])

        preds_inc_shuf3.append(p3_i_inc_fin)


        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

    
        preds_inc_shuf2.append(p2_i_inc_fin)

      else:
        
        latest_model_shuf1_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf1)

        loaded_model_shuf1_inc = model_b(verbose=VBS)

        loaded_model_shuf1_inc.load_weights(latest_model_shuf1_inc)

        loaded_model_shuf1_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf1_inc.fit(sple_rshp, p1_i_inc, callbacks=[cp_callback_model_shuf1])

        preds_inc_shuf1.append(p1_i_inc_fin)


        latest_model_shuf2_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf2)

        loaded_model_shuf2_inc = model_b(verbose=VBS)

        loaded_model_shuf2_inc.load_weights(latest_model_shuf2_inc)

        loaded_model_shuf2_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf2_inc.fit(sple_rshp, p2_i_inc, callbacks=[cp_callback_model_shuf2])

        preds_inc_shuf2.append(p2_i_inc_fin)


        latest_model_shuf3_inc=tf.train.latest_checkpoint(checkpoint_dir_model_shuf3)

        loaded_model_shuf3_inc = model_b(verbose=VBS)

        loaded_model_shuf3_inc.load_weights(latest_model_shuf3_inc)

        loaded_model_shuf3_inc.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        loaded_model_shuf3_inc.fit(sple_rshp, p3_i_inc, callbacks=[cp_callback_model_shuf3])


        preds_inc_shuf3.append(p3_i_inc_fin)


      mode_i_inc=mode([p1_i_inc_fin,p2_i_inc_fin,p3_i_inc_fin])
      m_vote_i_inc=mode_i_inc[0][0]
      m_vote_inc.append(m_vote_i_inc)

      mode_i=mode([p1_i_fin,p2_i_fin,p3_i_fin])
      m_vote_i=mode_i[0][0]
      m_vote.append(m_vote_i)

    batch_df_cp=batch_df.copy()
    batch_df_cp['sleep_stage_voted']=m_vote_inc
    batch_df_x=batch_df.copy()

    batch_df_x_rshp=np.array(batch_df_x).reshape(len(batch_df),3000,1)

    batch_df_y_=np_utils.to_categorical(m_vote_inc)

    teacher_model_i=model_b(verbose=VBS) #50 epochs teacher model

    teacher_model_i.load_weights(teacher_checkpt_path)
    teacher_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    teacher_model_i.fit(batch_df_x_rshp, batch_df_y_, epochs=50, callbacks=[cp_callback_teacher_model_i]) #write code in the previous cell to checkpoint every teacher model that is trained to every chunk of the batch_df data frame
    #fit the teacher model according to the batch df chunk and check point it

    student_model_i=get_model_1d_cnn()
    student_model_i.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])



    # Initialize and compile distiller
    distiller_i = Distiller(student=student_model_i, teacher=teacher_model_i)
    distiller_i.compile(optimizer=keras.optimizers.Adam(),
                      metrics=[keras.metrics.CategoricalAccuracy()],
                      student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True), #perform kd on the student model
                      distillation_loss_fn=keras.losses.KLDivergence(),
                      alpha=0.9,temperature=5)

    # Distill teacher to student
    #distiller.fit(x_dt_rshp, y_dt_, epochs=60, callbacks=[cp_callback_teacher_model_60_epochs])
    distiller_i.fit(batch_df_x_rshp, batch_df_y_, epochs=60)

    #call the batching() by passing batch_df_x




       

       
       #use the student model to perform preds on the batch df one at a time(go batck to the first line of the function)
       #check confidence, generate batch df....and continue until batch df is empty





















In [ ]:
student_preds=[]
indices=[]
for sple in np.array(x_test_dt[0:615]):

  sple_rshp=sple.reshape(1,3000,1)
  sple_df_rshp=sple.reshape(1,3000)
  student_i_probs=student_model.predict(sple_rshp)
  student_i_max_prob=student_i_probs.max()

  if student_i_max_prob < 0.7:

    batch_df=batch_df.append(pd.DataFrame(sple_df_rshp))
    indices=indices.append(sple_df_rshp.index)

  else:

    student_i_fin=np.argmax(student_i_probs,axis=1)

    student_preds.append(student_i_fin)

  #while batch_df.empty == False:

  if not batch_df.empty:

    len_df=len(batch_df)

    for sple in np.array(batch_df[0:615]):



